In [0]:
##### from __future__ import division
import sys
from ipyparallel import Client
import os, time
import pandas as pd
import numpy as np
import scipy as sp
from fractions import Fraction
from collections import defaultdict, OrderedDict, Counter
import vcf
import random
from geopy.distance import vincenty
import math
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.ticker as mtick
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.patches as mpatches
%matplotlib inline
from os import path as op
from os import listdir as ls
import shutil
from scipy.stats import hmean
from skbio.stats.distance import mantel


# Make SNPSFILE

In [0]:
stpDF = pd.read_csv('/home/lindb/wbp/sampsTOpop.txt',header=0,sep="\t")
stpDF.head()

In [0]:
pops = np.unique(stpDF['pop']).tolist()

In [0]:
popDict = OrderedDict()
for row in stpDF.index:
    pop = stpDF.loc[row,'pop']
    samp = stpDF.loc[row,'sampID']
    if not pop in popDict.keys():
        popDict[pop] = []
    popDict[pop].append(samp)
for pop in popDict.keys():
    print pop,len(popDict[pop])

In [0]:
%time data = pd.read_csv('/home/lindb/wbp/bayenv2/imputed_z12_maf_swp_trans_z12.txt',header=0,index_col=0,sep="\t")

In [0]:
data.head()

In [0]:
len(data.columns)

In [0]:
len(data.index)

In [0]:
popDictImp = OrderedDict()
for pop in sorted(pops):
    popDictImp[pop]  = data[data.index.isin(popDict[pop])]

In [0]:
total  = 0
for pop in popDictImp.keys():
    print pop, len(popDictImp[pop])
    total = total + len(popDictImp[pop])
print total

In [0]:
popDictImp['Dicks_Pass']['NODE_1000013_length_91_cov_1.802198_37'].tolist().count(0)

In [0]:
#just a test
locCount = 0
bayDict = OrderedDict()
for locus in data.columns:
    bayDict[locus] = OrderedDict()
    popCount = 0
    for pop in sorted(pops):
        
        zero = popDictImp[pop][locus].tolist().count(0) #count the first homozygotes
        one = popDictImp[pop][locus].tolist().count(1) #count the heterozygotes
        two = popDictImp[pop][locus].tolist().count(2) #count the second homozygotes        
        
        A1 = 2*zero + one #count of global major allele
        A2 = 2*two + one #count of global minor allele
        
        if len(bayDict[locus].keys()) == 0:
            bayDict[locus]['A1'] = OrderedDict()
            bayDict[locus]['A2'] = OrderedDict()
        bayDict[locus]['A1'][pop] = A1
        bayDict[locus]['A2'][pop] = A2
        #break
    locCount += 1
    if locCount % 10 == 0:
        print locCount
        break 
    #break

In [0]:
for locus in bayDict.keys():
    count = 0
    for allele in bayDict[locus].keys():
        for pop in bayDict[locus][allele].keys():
            count = count + bayDict[locus][allele][pop]
    print count

In [0]:
#make sure allele counts are correct
filE = '/home/lindb/wbp/bayenv2/testUnbinnedImputedSNPSFILE.txt'
with open(filE, 'w') as o:
    line = '\t'.join([str(pop) for pop in bayDict[bayDict.keys()[0]][bayDict[bayDict.keys()[0]].keys()[0]].keys()]) + str('\n')
    o.write("%s" % line)
    for locus in bayDict.keys():
        for allele in bayDict[locus].keys():
            line = str(locus) + '\t' + '\t'.join([str(x) for x in bayDict[locus][allele].values()]) + str('\n')
            #print locus,allele,line
            o.write("%s" % line)

In [0]:
#because it's way faster this way
text = '''from __future__ import division
import sys
from ipyparallel import Client
import os, time
import pandas as pd
import numpy as np
import scipy as sp
from fractions import Fraction
from collections import defaultdict, OrderedDict
import vcf
import random

filE1 = '/home/lindb/wbp/bayenv2/update.txt'
with open(filE1,'w') as o:
    text = 'reading stpDF\\n'
    o.write("%s" % text)
    
stpDF = pd.read_csv('/home/lindb/wbp/sampsTOpop.txt',header=0,sep="\\t")

pops = np.unique(stpDF['pop']).tolist()

popDict = OrderedDict()
for row in stpDF.index:
    pop = stpDF.loc[row,'pop']
    samp = stpDF.loc[row,'sampID']
    if not pop in popDict.keys():
        popDict[pop] = []
    popDict[pop].append(samp)

with open(filE1,'a') as o:
    text = 'reading data\\n'
    o.write("%s" % text)
    
data = pd.read_csv('/home/lindb/wbp/bayenv2/imputed_z12_maf_swp_trans_z12.txt',header=0,index_col=0,sep="\\t")

popDictImp = OrderedDict()
for pop in sorted(pops):
    popDictImp[pop]  = data[data.index.isin(popDict[pop])]
    
with open(filE1,'a') as o:
    text = 'iterating loci\\n'
    o.write("%s" % text)

locCount = 0
bayDict = OrderedDict()
for locus in data.columns:
    bayDict[locus] = OrderedDict()
    popCount = 0
    for pop in sorted(pops):
        
        zero = popDictImp[pop][locus].tolist().count(0) #count the first homozygotes
        one = popDictImp[pop][locus].tolist().count(1) #count the heterozygotes
        two = popDictImp[pop][locus].tolist().count(2) #count the second homozygotes        
        
        A1 = 2*zero + one
        A2 = 2*two + one
        
        if len(bayDict[locus].keys()) == 0:
            bayDict[locus]['A1'] = OrderedDict()
            bayDict[locus]['A2'] = OrderedDict()
        bayDict[locus]['A1'][pop] = A1
        bayDict[locus]['A2'][pop] = A2
        #break
    locCount += 1
    if locCount % 1000 == 0:
        print locCount
        with open(filE1,'a') as o:
            text = "%s\\n" % str(locCount)
            o.write("%s" % text)
        #break
    #break

filE = '/home/lindb/wbp/bayenv2/UnbinnedImputedSNPSFILE.txt'
with open(filE, 'w') as o:
    line = '\\t'.join([str(pop) for pop in bayDict[bayDict.keys()[0]][bayDict[bayDict.keys()[0]].keys()[0]].keys()]) + str('\\n')
    o.write("%s" % line)
    for locus in bayDict.keys():
        for allele in bayDict[locus].keys():
            line = str(locus) + '\\t' + '\\t'.join([str(x) for x in bayDict[locus][allele].values()]) + str('\\n')
            #print locus,allele,line
            o.write("%s" % line)
with open(filE1,'a') as o:
    text = 'Done!\\n'
    o.write("%s" % text)

'''

In [0]:
filE = '/home/lindb/wbp/bayenv2/get_SNPSFILE.py'
with open(filE,'w') as o:
    o.write("%s" % text)

In [0]:
shtext = '''#!/bin/bash
#$ -N snpsfile
#$ -V
#$ -j y
#$ -cwd

cd /home/lindb/wbp/bayenv2/
python get_SNPSFILE.py

'''
filE = '/home/lindb/wbp/bayenv2/get_SNPSFILE.sh'
with open(filE,'w') as o:
    o.write("%s" % shtext)

In [0]:
!qsub /home/lindb/wbp/bayenv2/get_SNPSFILE.sh

In [0]:
impbayenv = pd.read_csv('/home/lindb/wbp/bayenv2/UnbinnedImputedSNPSFILE.txt',header=0,index_col=0,sep="\t")
impbayenv.head()

In [0]:
impbayenv.to_csv('/home/lindb/wbp/bayenv2/UnbinnedImputedSNPSFILE_NOHEADERIDX.txt',header=False,index=False,sep="\t")

# estimate MATRIXFILE

In [0]:
import random

In [0]:
random.randint(100,100000)

In [0]:
shtext = '''#!/bin/bash
#$ -N matrixfile
#$ -V
#$ -j y
#$ -cwd

cd /home/lindb/wbp/bayenv2/
./bayenv2 -i UnbinnedImputedSNPSFILE_NOHEADERIDX.txt -p 8 -k 100000 -r %d > matrix.out

''' % random.randint(100,100000)
filE = '/home/lindb/wbp/bayenv2/get_MATRIXFILE.sh'
with open(filE,'w') as o:
    o.write("%s" % shtext)

In [0]:
!qsub /home/lindb/wbp/bayenv2/get_MATRIXFILE.sh

# make ENVIRONFILE

In [0]:
test = pd.read_csv('/home/lindb/wbp/WBP_IDS_MATCHED_POP_FINAL_02162016.csv',header=0,sep="\t")
test.head()

In [0]:
len(test.columns[10:-11])

In [0]:
test.columns

In [0]:
test.columns[8:-11]

In [0]:
envdf = pd.DataFrame(data=[],columns=sorted(pops))
envCount = 0
for col in test.columns[8:-11]: #for each environmental variable
    popMeans = [] #I'm averaging over the plots to get one measurement per population
    for pop in sorted(pops): #for each population
        data = test[test['Population_ID']==pop] # grab all data for samples in that pop
        popMean = np.mean(np.unique([x for x in data[col] if math.isnan(x) == False]).tolist())
        popMeans.append(popMean)

    mean = np.mean([float(x) for x in popMeans])
    std = np.std([float(x) for x in popMeans])
    est = [((x-mean)/(std)) for i,x in enumerate(popMeans)]
    envdf.loc[envCount,] = est
    envCount += 1
envdf.index = test.columns[8:-11]
envdf.sort_index(inplace=True)
envdf

In [0]:
#rename them for usage
envdf.index = ['AWS0-25',
 'AWS0-50',
 'Ann-ppt',
 'CEC',
 'Clay',
 'Elev',
 'GDD-Aug',
 'GDD-May',
 'Sand',
 'Silt',
 'Tmax-July',
 'Tmin-Jan',
 'WC15Bar',
 'WC3rdbar',
 'Lat',
 'Lon',
 'Max-rad-input',
 'Rock-cov']

In [0]:
envdf[''] = ""
envdf.sort_index(inplace=True)
envdf

In [0]:
envdf.to_csv('/home/lindb/wbp/bayenv2/ENVIRONFILE.txt',header=False,index=False,sep="\t")

In [0]:
envdf.to_csv('/home/lindb/wbp/bayenv2/ENVIRONFILE_headerIDX.txt',header=True,index=True,sep="\t")

In [0]:
envdf.head()

# make usable matrix file

In [0]:
matrixfile = open('/home/lindb/wbp/bayenv2/matrix.out')
r = matrixfile.readlines()
usable = '/home/lindb/wbp/bayenv2/usable_matrix.txt'
with open(usable,'w') as o:
    for line in r[2005:][:-1]:
        o.write(line)

In [0]:
umat = pd.read_csv(usable,header=None,sep="\t")
umat.shape

In [0]:
umat.loc[:,:7].to_csv(usable,header=False,index=False,sep="\t")

In [0]:
umat = umat.loc[:,:7]
umat.shape

# make bayenv .sh files - 1st chain

In [0]:
#we did a runthrough once (code in this section) and found that one chain isn't enough and can give spurious results 
    #depending on the seed given
#below this section here we've written code to do the remaining 4 chains

In [0]:
import shutil

In [0]:
snpdir = '/home/lindb/wbp/bayenv2/snps/'
if not os.path.exists(snpdir):
    os.makedirs(snpdir)
    shutil.copy('/home/lindb/wbp/bayenv2/bayenv2',os.path.join(snpdir,'bayenv2'))
    shutil.copy('/home/lindb/wbp/bayenv2/ENVIRONFILE.txt',os.path.join(snpdir,'ENVIRONFILE.txt'))
    shutil.copy('/home/lindb/wbp/bayenv2/UnbinnedImputedSNPSFILE_NOHEADERIDX.txt',
                os.path.join(snpdir,'UnbinnedImputedSNPSFILE_NOHEADERIDX.txt'))
    shutil.copy('/home/lindb/wbp/bayenv2/usable_matrix.txt',os.path.join(snpdir,'usable_matrix.txt'))

In [0]:
snpsfile = pd.read_csv('/home/lindb/wbp/bayenv2/UnbinnedImputedSNPSFILE.txt',header=0,index_col=0,sep="\t")
snpsfile.head()

In [0]:
#make SNPFILEs by using the SNP as the name of the file - will be easier reading the bf.environ.ENVIRONFILE.txt
df = snpsfile
for j in range(0,len(df.index),2):
    DF = df.loc[df.index[j],:]
    dfile = os.path.join(snpdir,df.index[j])
    DF.to_csv(dfile,header=False,index=False,sep="\t")

In [0]:
bayenv = os.listdir(snpdir)
bayenv = sorted([os.path.join(snpdir,f) for f in bayenv if 'NODE' in f])
len(bayenv)

In [0]:
bayenv_jobs = 190

In [0]:
num_chunks = int(np.round(len(bayenv)/bayenv_jobs))

In [0]:
num_chunks

In [0]:
chunks=[bayenv[x:x+num_chunks] for x in range(0, len(bayenv), num_chunks)]
chunks[0]

In [0]:
os.path.dirname(bayenv[0])

In [0]:
bayenv_exe = "/home/lindb/wbp/bayenv2/bayenv2"
bayenv_opt = "-i /home/lindb/wbp/bayenv2/UnbinnedImputedSNPSFILE_NOHEADERIDX.txt -m \
/home/lindb/wbp/bayenv2/usable_matrix.txt -e \
/home/lindb/wbp/bayenv2/ENVIRONFILE.txt \
-p 8 -k 100000 -n 18 -t -X -c -f -r placeholder"

#-t = test mode - calcs Z,BF, or rho for single snps
#-X = calc XTX
#-c = calculate rho in addition to BF
#-f = calc stand'd allele freqs
#-


total = 0
cpu = 0
max_cpu = 30
for i, chunk in enumerate(chunks):
    with open(os.path.join(os.path.dirname(bayenv[0]), "bayenv_parallel_%s" % str(i).zfill(3)), "w") as o:
        text = 'cd /home/lindb/wbp/bayenv2/snps'
        o.write("%s" % text)
        for bayenv_file in chunk:
            bayenv_cmd = " ".join([bayenv_exe, bayenv_opt]).split()
            if cpu == max_cpu:
                cpu = 0
            
            bayenv_cmd[2] = os.path.join('/home/lindb/wbp/bayenv2/snps',bayenv_file)
            bayenv_cmd[-1] = int(random.getrandbits(16))
            bayenv_cmd.insert(0, "taskset -c %d" % cpu)
            #bayenv_cmd.append("-o bayenv_%d" % i)
            bayenv_cmd.append("-o /home/lindb/wbp/bayenv2/bf_environfile.ENVIRONFILE.txt")
            o.write("%s\n" % " ".join([str(x) for x in bayenv_cmd]))
            total += 1
            cpu += 1

In [0]:
len(chunks)

In [0]:
total

In [0]:
rc = Client(profile='xmn')
dview = rc[:]
lview = rc.load_balanced_view()
len(dview)

In [0]:
def get_line_count(f):
    res = !wc -l {f}
    return int(res[2].split()[0])
dview['get_line_count'] = get_line_count

In [0]:
bay_par = os.listdir(snpdir)
bay_par = [os.path.join(snpdir,f) for f in bay_par if 'parallel' in f]
len(bay_par)

In [0]:
total = 0
for f in bay_par:
    print(f)
    total += get_line_count(f)
total

In [0]:
total

In [0]:
bay_par = sorted(bay_par)

In [0]:
bay_par[0:10]

In [0]:
def write_qsub_files(bayenv_parallel):
    files = []
    for i, f in enumerate(bayenv_parallel):
        d = '/home/lindb/wbp/bayenv2/qsubs/'
        if not os.path.exists(d):
            os.makedirs(d)
        qsub_file = os.path.join(d, "qsub_%s.sh" % str(i).zfill(3))
        files.append(qsub_file)
        with open(qsub_file, "w") as o:
            os.chmod(o.name, 0o744)
            print(o.name)
            o.write("%s\n" % "\n".join(["#!/bin/bash", 
                                        "#$ -N bayenv%d" % i,
                                        "#$ -V",
                                        "#$ -cwd",
                                        "#$ -pe smp 30",
                                        "#$ -j y",
                                        "#$ -q all.q",
                                        "unset module",
                                        "echo \"Running on $HOSTNAME\"",
                                        "cat %s | ~/bin/parallel -j 30 --progress --" % f]))
    return files
qsub_files = write_qsub_files(bay_par)

In [0]:
d = '/home/lindb/wbp/bayenv2/'
with open(os.path.join(d, "qsub_runner.sh"), "w") as o:
    os.chmod(o.name, 0o744)
    o.write("#!/bin/bash\n")
    o.write("unset module\n")
    for q in qsub_files:
        o.write("qsub %s\n" % q)

In [0]:
hey = !qhost | grep godel
qhost = hey[2:]

In [0]:
qhost[0].split()[0]

In [0]:
files = os.listdir('/home/lindb/wbp/bayenv2/snps/')
len(files)

In [0]:
for f in files:
    if not 'NODE' in f:
        print f

In [0]:
fs = [f for f in files if not 'NODE' in f and 'bay' not in f]
len(fs)

In [0]:
fs

In [0]:
from os import listdir as ls

In [0]:
tests = [f for f in ls('/home/lindb/wbp/bayenv2/snps/') if not 'freqs' in f]
len(tests)

In [0]:
freq = [f for f in ls('/home/lindb/wbp/bayenv2/snps/') if 'freqs' in f]
len(freq)

In [0]:
nodes = [f for f in tests if 'NODE' in f]
sec = [f for f in tests if 'NODE' not in f]
len(nodes),len(sec)

In [0]:
tests2 = [f for f in ls('/home/lindb/wbp/bayenv2/snps2/') if 'freqs' in f]
len(tests2)

In [0]:
tests2

In [0]:
104452+11730

In [0]:
len(list(set(tests2) - set(freq)))

# do remaining 4 chains

In [0]:
#get the 116231 SNPFILEs
DIR = '/home/lindb/wbp/bayenv2/snps/'
theSNPFILEs = [op.join(DIR,snp) for snp in ls(DIR) if 'NODE' in snp and 'freqs' not in snp]
len(theSNPFILEs)

In [0]:
op.basename(theSNPFILEs[0])

In [0]:
#make directories for the 4 chains
for i in range(4):
    num = i+2 #so we can consider our original run the 1st chain
    DIR = '/home/lindb/wbp/bayenv2/chain_%s' % str(num)
    print num
    
    #make a DIR for each chain and copy relevant files
    if not op.exists(DIR):
        os.makedirs(DIR)
    shutil.copy('/home/lindb/wbp/bayenv2/bayenv2',os.path.join(DIR,'bayenv2'))
    shutil.copy('/home/lindb/wbp/bayenv2/ENVIRONFILE.txt',os.path.join(DIR,'ENVIRONFILE.txt'))
    shutil.copy('/home/lindb/wbp/bayenv2/UnbinnedImputedSNPSFILE_NOHEADERIDX.txt',
                os.path.join(DIR,'UnbinnedImputedSNPSFILE_NOHEADERIDX.txt'))
    shutil.copy('/home/lindb/wbp/bayenv2/usable_matrix.txt',os.path.join(DIR,'usable_matrix.txt'))    
    
    
    #copy all of the SNPFILEs to the chain's snpdir
    print "copying SNPFILEs"
    snpdir = op.join(DIR,'snps')
    count = 0
    if i == 0: #just do this once,
        #make a DIR for SNPFILEs
        if not op.exists(snpdir):
            os.makedirs(snpdir)
        
        #copy the SNPFILEs
        for snp in theSNPFILEs:
            shutil.copy(snp,op.join(snpdir,op.basename(snp)))
            count += 1
            if count % 10000 == 0: #so I know progress
                print i, count
    else: #just copy the directory from the first chain
        shutil.copytree('/home/lindb/wbp/bayenv2/chain_2/snps',snpdir)

In [0]:
#get a list of all of the SNPFILEs across chains
allsnps = []
for i in range(4):
    num = i+2
    DIR = '/home/lindb/wbp/bayenv2/chain_%s/snps' % num
    if len(allsnps) == 0:
        allsnps = [op.join(DIR,snp) for snp in ls(DIR)]
    else:
        allsnps.extend([op.join(DIR,snp) for snp in ls(DIR)])
len(allsnps) #116231*4 = 464,924

In [0]:
allsnps[0]

In [0]:
allsnps[200000]

In [0]:
#set limit to number of cores requested
bayenv_jobs = 192#cpus

In [0]:
#devide the 4chains among the jobs
num_chunks = int(math.ceil((len(allsnps))/bayenv_jobs)) 
num_chunks

In [0]:
chunks=[allsnps[x:x+num_chunks] for x in range(0, len(allsnps), num_chunks)]
len(chunks)

In [0]:
len(chunks[0])

In [0]:
len(chunks[190])

In [0]:
len(chunks[191])

In [0]:
total = 0
for chunk in chunks:
    total = total + len(chunk)
total

In [0]:
#make a dir for the chunks
DIR = '/home/lindb/wbp/bayenv2/chunks'
os.makedirs(DIR)

In [0]:
op.dirname(op.dirname(SNPFILE))

In [0]:
1+1

In [0]:
len(chunks)

In [0]:
op.basename(exeDIR)

In [0]:
total = 0
cpu = 0
max_cpu = 32
for i,chunk in enumerate(chunks):
    print i
    with open(op.join('/home/lindb/wbp/bayenv2/chunks',"bayenv_parallel_%s" % str(i).zfill(3)), 'w') as o:
        for SNPFILE in chunk:
            if cpu == max_cpu: #reset if necessary
                cpu = 0
            
            #get path where SNPFILEs are
            snpdir = op.dirname(SNPFILE)
            
            #get folder of ./bayenv2, MATRIXFILE, ENVIRONFILE, and future outfile
            exeDIR = op.dirname(snpdir)
            
            #set options
            bayenv2_exe = op.join(exeDIR,'bayenv2')
            MATRIXFILE = op.join(exeDIR,'usable_matrix.txt')
            ENVIRONFILE = op.join(exeDIR,'ENVIRONFILE.txt')
            outfile = op.join(exeDIR,'%s_outfile' % op.basename(exeDIR))
            bay_cmd = '''%s \
%s \
-i %s \
-m %s \
-e %s \
-r %s \
-o %s \
-p 8 -k 100000 -n 18 -t -X -c -f\n''' % ("taskset -c %d" % cpu,
                                         bayenv2_exe,
                                         SNPFILE,
                                         MATRIXFILE,
                                         ENVIRONFILE,
                                         int(random.getrandbits(16)),
                                         outfile
                                        )
            total += 1
            cpu += 1
            #print SNPFILE
            #print bay_cmd
            o.write("%s" % bay_cmd)

In [0]:
DIR = '/home/lindb/wbp/bayenv2/chunks'
par_files = [op.join(DIR,f) for f in ls(DIR)]
len(par_files)

In [0]:
for p in sorted(par_files):
    text ='''#!/bin/bash
#$ -N chains%s
#$ -V
#$ -cwd
#$ -pe smp 30
#$ -j y
#$ -q all.q
unset module
echo "Running on $HOSTNAME"
cat %s | ~/bin/parallel -j 30 --progress --
''' % (str(p[-3:]),
       p
      )
    filE = '/home/lindb/wbp/bayenv2/chunks/qsubs/qsub_%s.sh' % str(p[-3:])
    print text
    with open(filE,'w') as o:
        os.chmod(o.name, 0o744)
        o.write("%s" % text)    

# finish the rest of the jobs

In [0]:
snpsfile.head()

In [0]:
snpset = snpsfile.index.tolist()

In [0]:
chains = [chain for chain in ls('/home/lindb/wbp/bayenv2/') if 'chain' in chain]
len(chdirs)

In [0]:
chains

In [0]:
redoDict = OrderedDict()
for chain in chains:
    DIR = op.join('/home/lindb/wbp/bayenv2/',chain)
    bffile = op.join(DIR,'%s_outfile.bf' % chain)
    bfdf = pd.read_csv(bffile,header=None,sep='\t')
    print chain,len(bfdf.index),len(np.unique(bfdf[0].tolist()).tolist())
    loci = [op.basename(path) for path in bfdf[0]]
    
    redoDict[chain] = list( set(snpset) - set(loci))

In [0]:
redoDict['chain_5']

In [0]:
for chain in chains:
    filE = '/home/lindb/wbp/bayenv2/%s/redos.txt' % chain
    with open(filE,'w') as o:
        for locus in redoDict[chain]:
            text = '''/home/lindb/wbp/bayenv2/bayenv2 -i /home/lindb/wbp/bayenv2/%s/snps/%s \
-m /home/lindb/wbp/bayenv2/usable_matrix.txt \
-e /home/lindb/wbp/bayenv2/ENVIRONFILE.txt \
-r %d \
-o %s \
-p 8 -k 100000 -n 18 -t -X -c -f
''' % (chain,
       locus,
       int(random.getrandbits(16)),
       op.join(op.dirname(filE),'%s_outfile' % chain)
      )
            o.write("%s" % text)

# are there environmental clines? (related to geographic distance)

### these relationships are better explored in the 'allele frequency shifts' section

In [0]:
envdf = envdf.loc[:,[col for col in envdf.columns[:-1]]]
envdf.head()

In [0]:
#double check each row has mean =0 , stdev=1.0
for row in envdf.index:
    print row,np.mean(envdf.loc[row,:]),np.std(envdf.loc[row,:])

In [0]:
envdf.head()

In [0]:
#get pairwise environmental distances across all environmental variables
envdifDict = OrderedDict()
for i,popi in enumerate(envdf.columns):
    envdifDict[popi] = OrderedDict()
    for j,popj in enumerate(envdf.columns):
        if i < j: #lower tri - reverse logic when put into dataframe
            sqdif = []
            for envf in envdf.index:
                xi = envdf.loc[env,popi]
                xj = envdf.loc[env,popj]
                difsq = (xi - xj)**2
                sqdif.append(difsq)
            sums = sum(sqdif)
            envdifDict[popi][popj] = np.sqrt(sums)
        else:
            envdifDict[popi][popj] = np.nan
envdist = pd.DataFrame(envdifDict)
envdist

In [0]:
filE = '/home/lindb/wbp/bayenv2/matrices/environmental_distances.txt'
envdist.to_csv(filE,header=True,index=True,sep='\t')

In [0]:
test.head()

In [0]:
#get pop coordinates
pops = np.unique(test['Population_ID']).tolist()
popDict = OrderedDict()
for pop in pops:
    data = test[test['Population_ID'] == pop]
    ulat = [lat for lat in np.unique(data['lat']).tolist() if math.isnan(lat) == False]
    ulon = [lon for lon in np.unique(data['lon']).tolist() if math.isnan(lon) == False]
    mlat = np.mean(ulat)
    mlon = np.mean(ulon)
    popDict[pop] = (mlat,mlon)

In [0]:
#get vincenty distances between populations to account for earth curvature
geoDict = OrderedDict()
for i,popi in enumerate(envdf.columns):
    geoDict[popi] = OrderedDict()
    for j,popj in enumerate(envdf.columns):
        if i < j: #lower tri - reverse logic when put into dataframe
            geoDict[popi][popj] = vincenty(popDict[popi],popDict[popj])
        else:
            geoDict[popi][popj] = np.nan
geodist = pd.DataFrame(geoDict)
geodist

In [0]:
filE = '/home/lindb/wbp/bayenv2/matrices/geographic_distances.txt'
geodist.to_csv(filE,header=True,index=True,sep='\t')

In [0]:
envs = []
for i,popi in enumerate(envdist.index):
    for j,popj in enumerate(envdist.index):
        if i > j:
            envs.append(envdist.loc[popi,popj])
envs

In [0]:
geos = []
for i,popi in enumerate(geodist.index):
    for j,popj in enumerate(geodist.index):
        if i > j:
            geos.append(float(str(geodist.loc[popi,popj]).split(" ")[0]))
geos

In [0]:
spearmanr(envs,geos)

In [0]:
plt.scatter(envs,geos)

### what about for envs which we expect to be clinal?

In [0]:
envdf.index[2]

In [0]:
#get pairwise environmental distances
precipDict = OrderedDict()
for i,popi in enumerate(envdf.columns):
    precipDict[popi] = OrderedDict()
    for j,popj in enumerate(envdf.columns):
        if i < j: #lower tri - reverse logic when put into dataframe
            sqdif = []
            for env in [envdf.index[2]]: #just for Ann-ppt
                xi = envdf.loc[env,popi]
                xj = envdf.loc[env,popj]
                difsq = (xi - xj)**2
                sqdif.append(difsq)
            sums = sum(sqdif)
            precipDict[popi][popj] = np.sqrt(sums)
        else:
            precipDict[popi][popj] = np.nan
precipdist = pd.DataFrame(precipDict)
precipdist

In [0]:
sqdif

In [0]:
precips = []
for i,popi in enumerate(precipdist.index):
    for j,popj in enumerate(precipdist.index):
        if i > j:
            precips.append(precipdist.loc[popi,popj])
precips

In [0]:
spearmanr(precips,geos)

In [0]:
plt.scatter(precips,geos)

In [0]:
plt.scatter(envs,geos)

In [0]:
plt.scatter(envs,precips)

# get harmonic mean of BFs across chains

In [0]:
envDict['Ann-ppt']

In [0]:
#get a dictionary to name columns in bf_environfile.ENVIRONFILE
orderDict = {}
orderDict[0] = 'BF'
orderDict[1] = 'rho'
orderDict[2] = 'pearson'
colDict = OrderedDict()
for env,cols in envDict.items():
    #print env,cols
    colCount = 0
    for col in cols:
        sub = '_'.join([str(env), orderDict[colCount]]) 
        colDict[col] = sub
        colCount += 1

In [0]:
chains = sorted([chain for chain in ls('/home/lindb/wbp/bayenv2/') if 'chain' in chain])
chains

In [0]:
#load and transform the outfile to have indices and column names
#put df into a dictionary based on chain ID
outDict = OrderedDict()
for chain in chains:
    print chain
    bff = '/home/lindb/wbp/bayenv2/%s/%s_outfile.bf' % (chain,chain)
    bfdf = pd.read_csv(bff,header=None,sep='\t')
    loci = []
    for row in bfdf.index:
        locus = bfdf.loc[row,0].split("/")[-1]
        loci.append(locus)
    outdf = bfdf.loc[:,[x for x in bfdf.columns[1:-1]]]
    outdf.columns = [col for col in range(len(outdf.columns))]
    outdf.columns = [colDict[col] for col in outdf.columns]
    outdf.index = [locus for locus in loci]
    outDict[chain] = outdf

In [0]:
outDict['chain_5'].head()

In [0]:
outDict['chain_5'].shape

In [0]:
#create a dictionary which has BFs separated by environment
bfDict = OrderedDict() #bfDict[env] = dataframe, cols = chain1-5 for a single env
for env in envDict.keys():
    print env
    for c,chain in enumerate(outDict.keys()):
        if c == 0:
            df = outDict[chain].loc[:,[col for col in outDict[chain].columns if ('%s_BF' % env) in col]]
            df.columns = ['%s_BF_%s' % (env,chain)]
        else:
            new = outDict[chain].loc[:,[col for col in outDict[chain].columns if ('%s_BF' % env) in col]]
            new.columns = ['%s_BF_%s' % (env,chain)]
            df = pd.merge(df,new,left_index=True,right_index=True)
    bfDict[env] = df

In [0]:
from scipy.stats import variation as spvar

In [0]:
statsDict = OrderedDict()
for env in bfDict.keys():
    print env
    statsDict[env] = OrderedDict() #keys = coef (std/mean), |max - min|
    statsDict[env]['coef'] = []
    statsDict[env]['abs_diff'] = []
    
    for locus in bfDict[env].index:
        data = bfDict[env].loc[locus,:].tolist()
        coef = spvar(data)
        absdiff = abs(max(data) - min(data))
        statsDict[env]['coef'].append(coef)
        statsDict[env]['abs_diff'].append(absdiff)
    #print pd.Series(statsDict[env]['abs_diff']).describe()
    #plt.hist(statsDict[env]['abs_diff'],bins = np.arange(0,max(statsDict[env]['abs_diff']),10))[2]
    
    #print pd.Series(statsDict[env]['coef']).describe()
    #plt.hist(statsDict[env]['coef'],bins = np.arange(0,max(statsDict[env]['coef']),0.05))

In [0]:
plt.hist(statsDict['AWS0-25']['coef'],bins = np.arange(-0.05,max(statsDict['AWS0-25']['coef']),0.05))[2]

In [0]:
plt.hist(statsDict['AWS0-25']['abs_diff'],bins = np.arange(0,1400,10))

In [0]:
pd.Series(statsDict['AWS0-25']['abs_diff']).describe()

In [0]:
idx = []
for i,x in enumerate(statsDict['AWS0-25']['abs_diff']):
    if x > 18:
        idx.append(i)

In [0]:
bfDict['AWS0-25'].loc[bfDict['AWS0-25'].index[[x for x in idx]],:]
#NODE_280450_length_96_cov_2.343750_15

In [0]:
hmean([450,580,400,425,0.1]),hmean([450,580,400,425]),np.mean([450,580,400,425,0.1]),np.mean([450,580,400,425])

In [0]:
hmean([.045,.0580,.0400,.0425,0.1]),hmean([.0450,.0580,.0400,.0425]),np.mean([.0450,.0580,.0400,.0425,0.1]),
np.mean([.0450,.0580,.0400,.0425])

In [0]:
hmean([62.197,0.083,.362,.054,.078]),np.mean([62.197,0.083,.362,.054,.078])

In [0]:
#calculate harmonic mean, flag SNPs with >=2 chains with BF>1
harmDict= OrderedDict()
for env in bfDict.keys():
    print env
    harmDict[env] = OrderedDict()
    for locus in bfDict[env].index:
        data = bfDict[env].loc[locus,:].tolist()
        bigs = [x for x in data if x>1]
        if len(bigs) >2:
            print bigs
            harmDict[env][locus] = hmean(bigs)
        else:
            good = [x for x in data if x not in bigs]
            harmDict[env][locus] = hmean(good)
    print sorted(harmDict[env].values())[-10:]

In [0]:
#write out hmean for each snp to file
for env in harmDict.keys():
    print env
    filE = '/home/lindb/wbp/bayenv2/results/%s_all_BFs.txt' % env
    with open(filE,'w') as o:
        line = '\t'.join(['locus','BF']) + '\n'
        o.write("%s" % line)
        for locus in harmDict[env].keys():
            line = '\t'.join([str(locus),str(harmDict[env][locus])])+'\n'
            o.write("%s" % line)
harmdfDict = OrderedDict()
for env in harmDict.keys():
    filE = '/home/lindb/wbp/bayenv2/results/%s_BFs.txt' % env
    harmdfDict[env] = pd.read_csv(filE,header=0,sep='\t')
harmdfDict[env]

In [0]:
#do some sorting, rewrite the file
for env in harmdfDict.keys():
    harmdfDict[env].sort_values(by='BF',inplace=True)
    filE = '/home/lindb/wbp/bayenv2/results/%s_BFs.txt' % env
    harmdfDict[env].to_csv(filE,header=True,index=False,sep='\t')

In [0]:
#in case I have to restart my notebook
harmdfDict = OrderedDict()
for env in harmDict.keys():
    filE = '/home/lindb/wbp/bayenv2/results/%s_BFs.txt' % env
    harmdfDict[env] = pd.read_csv(filE,header=0,sep='\t')

In [0]:
harmdfDict[env]

# get harmonic mean of abs rho across chains

In [0]:
#create a dictionary which has Rho's separated by environment
rhoDict = OrderedDict() #rhoDict[env] = dataframe, cols = chain1-5 for a single env
for env in envDict.keys():
    print env
    for c,chain in enumerate(outDict.keys()):
        if c == 0:
            df = outDict[chain].loc[:,[col for col in outDict[chain].columns if ('%s_rho' % env) in col]]
            df.columns = ['%s_rho_%s' % (env,chain)]
        else:
            new = outDict[chain].loc[:,[col for col in outDict[chain].columns if ('%s_rho' % env) in col]]
            new.columns = ['%s_rho_%s' % (env,chain)]
            df = pd.merge(df,new,left_index=True,right_index=True)
    rhoDict[env] = df

In [0]:
#get harmonic means of the absolute value of rho
hrhoDict= OrderedDict()
for env in rhoDict.keys():
    print env
    hrhoDict[env] = OrderedDict()
    for locus in rhoDict[env].index:
        data = rhoDict[env].loc[locus,:].tolist()
        data = [abs(x) for x in data if x != 0] #harmonic mean can't use 0s
        hrhoDict[env][locus] = hmean(data)
    print sorted(hrhoDict[env].values())[-10:]

In [0]:
#write to file
for env in hrhoDict.keys():
    print env
    filE = '/home/lindb/wbp/bayenv2/results/%s_all_rhos.txt' % env
    with open(filE,'w') as o:
        line = '\t'.join(['locus','rho']) + '\n'
        o.write("%s" % line)
        for locus in hrhoDict[env].keys():
            line = '\t'.join([str(locus),str(hrhoDict[env][locus])])+'\n'
            o.write("%s" % line)

In [0]:
#sort and rewrite to file
hrhodfDict = OrderedDict()
for env in hrhoDict.keys():
    filE = '/home/lindb/wbp/bayenv2/results/%s_all_rhos.txt' % env
    hrhodfDict[env] = pd.read_csv(filE,header=0,sep='\t')
    hrhodfDict[env].sort_values(by='rho',inplace=True)
    hrhodfDict[env].to_csv(filE,header=True,index=False,sep='\t')

## get loci in intersection of top BFs and top rhos

### top 0.5% BF (n=582), top 1%rho (n=1,163)

In [0]:
#get top rhos and BFs from dataframes for each env - since sorted by value, just the top % using index
topBFs  = OrderedDict()
topRhos = OrderedDict()
for env in harmdfDict.keys():
    topbfs = pd.DataFrame(harmdfDict[env].loc[harmdfDict[env].index[-582:],:])
    topBFs[env] = topbfs['locus'].tolist()
    
    toprhos = pd.DataFrame(hrhodfDict[env].loc[hrhodfDict[env].index[-1163:],:])
    topRhos[env] = toprhos['locus'].tolist()

In [0]:
#look at min and max values
minrho = 1
maxrho = -1
for env in harmdfDict.keys():
    topbfs = pd.DataFrame(harmdfDict[env].loc[harmdfDict[env].index[-582:],:])
    print env
    print 'BF =',round(min(topbfs['BF']),3),",",round(max(topbfs['BF']),3)
    
    toprhos = pd.DataFrame(hrhodfDict[env].loc[hrhodfDict[env].index[-1163:],:])
    print 'rho =',round(min(toprhos['rho']),3),",",round(max(toprhos['rho']),3)
    print '\n'
    if min(toprhos['rho']) < minrho:
        minrho = min(toprhos['rho'])
    if max(toprhos['rho']) > maxrho:
        maxrho = max(toprhos['rho'])
print 'min=',minrho
print 'max=',maxrho

In [0]:
#look at min and max values
minrho = 1
maxrho = -1
for env in harmdfDict.keys():
    topbfs = pd.DataFrame(harmdfDict[env].loc[harmdfDict[env].index[-582:],:])
    print env
    print 'BF =',min(topbfs['BF']),max(topbfs['BF'])
    
    toprhos = pd.DataFrame(hrhodfDict[env].loc[hrhodfDict[env].index[-1163:],:])
    print 'rho =',min(toprhos['rho']),max(toprhos['rho'])
    print '\n'
    if min(toprhos['rho']) < minrho:
        minrho = min(toprhos['rho'])
    if max(toprhos['rho']) > maxrho:
        maxrho = max(toprhos['rho'])
print 'min=',minrho
print 'max=',maxrho

In [0]:
toprhos.head()

In [0]:
#get robust SNPs via intersection of top BFs and top rhos
intset = OrderedDict()
for env in topRhos.keys():
    intset[env] = list(set(topBFs[env]).intersection(set(topRhos[env])))
    print env, len(intset[env])

In [0]:
lens = [len(intset[env]) for env in intset.keys()]
max(lens),min(lens),np.mean(lens)

In [0]:
#write list of robust SNPs to file
for env in intset.keys():
    df = pd.DataFrame(intset[env])
    df.columns = [str(env)]
    filE = '/home/lindb/wbp/bayenv2/results/sigsnps/%s_top_p05%%BF_1%%rho.txt' % env
    df.to_csv(filE,header=True,index=False,sep='\t')

In [0]:
#when I restart my notebook
DIR = '/home/lindb/wbp/bayenv2/results/sigsnps/'
files = [op.join(DIR,f) for f in ls(DIR)]
intset = {}
for f in files:
    env = op.basename(f).split("_")[0]
    df = pd.read_csv(f,header=0,sep='\t')
    intset[env] = df[env].tolist()

In [0]:
#check against previous
for env in sorted(intset):
    print env,len(intset[env])

# covariances

In [0]:
#file made in 09_OutFLANK.ipynb
impMAF = pd.read_csv('/home/lindb/wbp/OutFLANK/imputed_MAF.txt',header=0,index_col=0,sep="\t")
impMAF.head()

In [0]:
#count > 0.5
for pop in impMAF:
    df = pd.DataFrame(impMAF[pop][impMAF[pop]>0.5])
    print pop,len(df.index)

In [0]:
#check to make sure impMAF doesn't just have MAFs - impMAF = freq of global minor allele
for pop in impMAF:
    print pop,max(impMAF[pop])

In [0]:
#file made in 09_OutFLANK.ipynb
globmafs = pd.read_csv('/home/lindb/wbp/OutFLANK/global_mafs.txt',header=0,sep='\t')
globmafs.head()

In [0]:
len(globmafs.index)

In [0]:
#create dictionary, key = SNP, val = maf
glob = OrderedDict()
for row in globmafs.index:
    locus = globmafs.loc[row,'locus']
    glob[locus] = globmafs.loc[row,'maf']

In [0]:
from __future__ import division

In [0]:
#population sizes
popDict = {'Dicks_Pass':25,
'Freel_Peak':48,
'Little_Round_Top':25,
'Heavenly':25,
'Mt_Rose_Ophir':49,
'Rifle_Peak':24,
'Snow_Valley_Peak':24,
'West_Shore_Peaks':24}

In [0]:
sum(popDict.values())

In [0]:
#do pairwise to get Dij
dijDict = OrderedDict() 
for env in intset.keys():
    outliersnps = intset[env]
    
    dijDict[env] = OrderedDict() 
    icount = 0
    for i,locusi in enumerate(outliersnps):
        dijDict[env][locusi] = OrderedDict()
        qi = glob[locusi] #global maf

        for j,locusj in enumerate(outliersnps):
            if i > j: #i=row, j=col : lower triangle 
                qj = glob[locusj] #global maf

                sums = 0
                for pop in impMAF.columns:
                    qik = impMAF.loc[locusi,pop] #get pop af
                    qjk = impMAF.loc[locusj,pop] #get pop af
                    nk = popDict[pop]

                    sums += (nk/sum(popDict.values()))*((qik*qjk)-(qi*qj))

                dijDict[env][locusi][locusj] = sums
            else:
                dijDict[env][locusi][locusj] = np.nan
        icount += 1
        if icount % 10 == 0:
            print env,icount

In [0]:
#write out the file
for env in dijDict.keys():
    filE = '/home/lindb/wbp/bayenv2/covariances/dvals/%s_dvals.txt' % env
    with open(filE,'w') as o:
        key0 = dijDict[env].keys()[0]
        line = '\t'.join(dijDict[env][key0].keys()) + str('\n')
        o.write("%s" % line)
        for locusi in dijDict[env].keys():
            line = str(locusi)+'\t'+'\t'.join([str(x) for x in dijDict[env][locusi].values()]) + str('\n')
            o.write("%s" % line)

In [0]:
#check out a file
dvals = pd.read_csv(filE,header=0,index_col=0,sep="\t")
dvals.head()

In [0]:
1+1

In [0]:
#get H_exp for each snp
#file made in 09_OutFLANK.ipynb
filE = '/home/lindb/wbp/OutFLANK/Hexp_by_snp_withbins.txt'
H = pd.read_csv(filE,header=0,index_col=0,sep='\t')
H = pd.DataFrame(H.loc[:,['h_exp','bin']])
H.head()

In [0]:
#Hexp for the complete set of SNPs
plt.hist(H['h_exp'],bins = [x for x in np.arange(0,.51,0.01)])[2]

In [0]:
#get a dataframe with the outlier loci and their bins
outlierdata = OrderedDict()
for env in dijDict.keys():
    outliersnps = intset[env]

    outlierdata[env] = pd.DataFrame(H[H.index.isin(outliersnps)])

In [0]:
dijDict.keys()

In [0]:
with PdfPages('/home/lindb/wbp/figures/grid_imp_hexp_hist_by_env_AWSO-Lat.pdf') as pdf:
    plt.close('all')
    fig, ((a1,a2,a3),(a4,a5,a6),(a7,a8,a9)) = plt.subplots(3, 3, figsize=(5,5),dpi=400)

    a1 = plt.subplot(331)
    a1.hist(outlierdata['AWS0-25']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a1.set_title('A',y=.9,loc='right',fontsize=20,fontweight='bold')
    a1.spines['right'].set_visible(False)
    a1.spines['top'].set_visible(False)
    a1.yaxis.set_ticks_position('left')
    a1.xaxis.set_ticks_position('bottom')
    
    a2 = plt.subplot(332)
    a2.hist(outlierdata['AWS0-50']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a2.set_title('B',y=.9,loc='right',fontsize=20,fontweight='bold')
    a2.spines['right'].set_visible(False)
    a2.spines['top'].set_visible(False)
    a2.yaxis.set_ticks_position('left')
    a2.xaxis.set_ticks_position('bottom')
    
    a3 = plt.subplot(333)
    a3.hist(outlierdata['Ann-ppt']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a3.set_title('C',y=.9,loc='right',fontsize=20,fontweight='bold')
    a3.spines['right'].set_visible(False)
    a3.spines['top'].set_visible(False)
    a3.yaxis.set_ticks_position('left')
    a3.xaxis.set_ticks_position('bottom')
    
    a4 = plt.subplot(334)
    a4.hist(outlierdata['CEC']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a4.set_title('D',y=.9,loc='right',fontsize=20,fontweight='bold')
    a4.spines['right'].set_visible(False)
    a4.spines['top'].set_visible(False)
    a4.yaxis.set_ticks_position('left')
    a4.xaxis.set_ticks_position('bottom')
    
    a5 = plt.subplot(335)
    a5.hist(outlierdata['Clay']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a5.set_title('E',y=.9,loc='right',fontsize=20,fontweight='bold')
    a5.spines['right'].set_visible(False)
    a5.spines['top'].set_visible(False)
    a5.yaxis.set_ticks_position('left')
    a5.xaxis.set_ticks_position('bottom')
    
    a6 = plt.subplot(336)
    a6.hist(outlierdata['Elev']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a6.set_title('F',y=.9,loc='right',fontsize=20,fontweight='bold')
    a6.spines['right'].set_visible(False)
    a6.spines['top'].set_visible(False)
    a6.yaxis.set_ticks_position('left')
    a6.xaxis.set_ticks_position('bottom')
    
    a7 = plt.subplot(337)
    a7.hist(outlierdata['GDD-Aug']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a7.set_title('G',y=.9,loc='right',fontsize=20,fontweight='bold')
    a7.spines['right'].set_visible(False)
    a7.spines['top'].set_visible(False)
    a7.yaxis.set_ticks_position('left')
    a7.xaxis.set_ticks_position('bottom')
    
    a8 = plt.subplot(338)
    a8.hist(outlierdata['GDD-May']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a8.set_title('H',y=.9,loc='right',fontsize=20,fontweight='bold')
    a8.spines['right'].set_visible(False)
    a8.spines['top'].set_visible(False)
    a8.yaxis.set_ticks_position('left')
    a8.xaxis.set_ticks_position('bottom')
    
    a9 = plt.subplot(339)
    a9.hist(outlierdata['Lat']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a9.set_title('I',y=.9,loc='right',fontsize=20,fontweight='bold')
    a9.spines['right'].set_visible(False)
    a9.spines['top'].set_visible(False)
    a9.yaxis.set_ticks_position('left')
    a9.xaxis.set_ticks_position('bottom')
    
    fig.set_size_inches(13,10)
    pdf.savefig(fig,bbox_inches='tight')

In [0]:
with PdfPages('/home/lindb/wbp/figures/grid_imp_hexp_hist_by_env_lon-WC3rdbar.pdf') as pdf:
    plt.close('all')
    fig, ((a1,a2,a3),(a4,a5,a6),(a7,a8,a9)) = plt.subplots(3, 3, figsize=(5,5),dpi=400)

    a1 = plt.subplot(331)
    a1.hist(outlierdata['Lon']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a1.set_title('A',y=.9,loc='right',fontsize=20,fontweight='bold')
    a1.spines['right'].set_visible(False)
    a1.spines['top'].set_visible(False)
    a1.yaxis.set_ticks_position('left')
    a1.xaxis.set_ticks_position('bottom')
    
    a2 = plt.subplot(332)
    a2.hist(outlierdata['Max-rad-input']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a2.set_title('B',y=.9,loc='right',fontsize=20,fontweight='bold')
    a2.spines['right'].set_visible(False)
    a2.spines['top'].set_visible(False)
    a2.yaxis.set_ticks_position('left')
    a2.xaxis.set_ticks_position('bottom')
    
    a3 = plt.subplot(333)
    a3.hist(outlierdata['Rock-cov']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a3.set_title('C',y=.9,loc='right',fontsize=20,fontweight='bold')
    a3.spines['right'].set_visible(False)
    a3.spines['top'].set_visible(False)
    a3.yaxis.set_ticks_position('left')
    a3.xaxis.set_ticks_position('bottom')
    
    a4 = plt.subplot(334)
    a4.hist(outlierdata['Sand']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a4.set_title('D',y=.9,loc='right',fontsize=20,fontweight='bold')
    a4.spines['right'].set_visible(False)
    a4.spines['top'].set_visible(False)
    a4.yaxis.set_ticks_position('left')
    a4.xaxis.set_ticks_position('bottom')
    
    a5 = plt.subplot(335)
    a5.hist(outlierdata['Silt']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a5.set_title('E',y=.9,loc='right',fontsize=20,fontweight='bold')
    a5.spines['right'].set_visible(False)
    a5.spines['top'].set_visible(False)
    a5.yaxis.set_ticks_position('left')
    a5.xaxis.set_ticks_position('bottom')
    
    a6 = plt.subplot(336)
    a6.hist(outlierdata['Tmax-July']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a6.set_title('F',y=.9,loc='right',fontsize=20,fontweight='bold')
    a6.spines['right'].set_visible(False)
    a6.spines['top'].set_visible(False)
    a6.yaxis.set_ticks_position('left')
    a6.xaxis.set_ticks_position('bottom')
    
    a7 = plt.subplot(337)
    a7.hist(outlierdata['Tmin-Jan']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a7.set_title('G',y=.9,loc='right',fontsize=20,fontweight='bold')
    a7.spines['right'].set_visible(False)
    a7.spines['top'].set_visible(False)
    a7.yaxis.set_ticks_position('left')
    a7.xaxis.set_ticks_position('bottom')
    
    a8 = plt.subplot(338)
    a8.hist(outlierdata['WC15Bar']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a8.set_title('H',y=.9,loc='right',fontsize=20,fontweight='bold')
    a8.spines['right'].set_visible(False)
    a8.spines['top'].set_visible(False)
    a8.yaxis.set_ticks_position('left')
    a8.xaxis.set_ticks_position('bottom')
    
    a9 = plt.subplot(339)
    a9.hist(outlierdata['WC3rdbar']['h_exp'].tolist(),bins = [binn for binn in np.arange(0,0.51,0.01)])[2]
    a9.set_title('I',y=.9,loc='right',fontsize=20,fontweight='bold')
    a9.spines['right'].set_visible(False)
    a9.spines['top'].set_visible(False)
    a9.yaxis.set_ticks_position('left')
    a9.xaxis.set_ticks_position('bottom')
    
    fig.set_size_inches(13,10)
    pdf.savefig(fig,bbox_inches='tight')

In [0]:
#get sets of SNPs to make random draws
nonsigsnps = OrderedDict()
for env in outlierdata.keys():
    nonsigs = list(set(H.index.tolist()) - set(outlierdata[env].index.tolist()))
    nonsigsnps[env] = pd.DataFrame(H[H.index.isin(nonsigs)])
    print len(outlierdata[env].index.tolist())+len(nonsigsnps[env].index)

In [0]:
#how many random snps to draw from each bin?
binCounter = OrderedDict()
for env in outlierdata.keys():
    binCounter[env] = Counter()
    for row in outlierdata[env].index:
        binCounter[env][outlierdata[env].loc[row,'bin']] += 1
    for b in binCounter[env].keys():
        print env,b,binCounter[env][b]

In [0]:
binCounter['AWS0-25']

In [0]:
os.makedirs('/home/lindb/wbp/bayenv2/covariances/randmatrices')
os.makedirs('/home/lindb/wbp/bayenv2/covariances/randmatrices/randsnps')

In [0]:
#for each env make 1000 dataframes with a set of snps == len(outliersnps[env])
for env in outlierdata.keys():
    print env
    for i in range(1000):  
        snps = []        
        for binn in binCounter[env].keys():
            data = nonsigsnps[env][nonsigsnps[env]['bin'] == binn]
                
            [snps.append(snp) for snp in random.sample(data.index,binCounter[env][binn])]
        
        assert len(np.unique(snps).tolist()) == sum(binCounter[env].values())

        DIR = '/home/lindb/wbp/bayenv2/covariances/randmatrices/randsnps/%s' % env
        if not op.exists(DIR):
            os.makedirs(DIR)
        filE = op.join(DIR,"%s_%s_randsnps.txt" % (env,str(i).zfill(3)))
        df = pd.DataFrame(snps)
        df.to_csv(filE,header=False,index=False,sep="\t")

In [0]:
#quicker to do it this way
#get dvals for the sets of random snps for each env
for env in outlierdata.keys():
    for i in range(1000):
        text = '''from __future__ import division
import os
from collections import OrderedDict,Counter
import pandas as pd
import numpy as np
import random
import math
from os import path as op
from os import listdir as ls

#get pop assignment for each samp
filE = '/home/lindb/wbp/sampsTOpop.txt'
stp = pd.read_csv(filE,header=0,sep="\t")


#pops matched to samps
ptsDict = OrderedDict() #pop to samp dictionary
for row in stp.index:
    pop = stp.loc[row,'pop']
    if not pop in ptsDict.keys():
        ptsDict[pop] = []
    ptsDict[pop].append(stp.loc[row,'sampID'])


#get a dict with num indiv in pop
popDict = OrderedDict()
total = 0
for pop in ptsDict.keys():
    popDict[pop] = len(ptsDict[pop])
    print pop,popDict[pop]
    total += popDict[pop]


filE = '/home/lindb/wbp/OutFLANK/global_mafs.txt'
globs = pd.read_csv(filE,header=0,sep='\\t')
glob = OrderedDict()
for row in globs.index:
    snp = globs.loc[row,'locus']
    maf = globs.loc[row,'maf']
    glob[snp] = maf

impMAF = pd.read_csv('/home/lindb/wbp/OutFLANK/imputed_MAF.txt',header=0,index_col=0,sep="\\t")

filE= '/home/lindb/wbp/bayenv2/covariances/randmatrices/randsnps/%s/%s_%s_randsnps.txt' 
df = pd.read_csv(filE,header=None,sep="\\t")
randomsnps = df[0].tolist()

dijDict = OrderedDict() 
for i,locusi in enumerate(randomsnps):
    dijDict[locusi] = OrderedDict()
    qi = glob[locusi] #global af

    for j,locusj in enumerate(randomsnps):
        if i > j: #i=row, j=col : lower triangle 
            qj = glob[locusj] #global af

            sums = 0
            for pop in impMAF.columns:
                qik = impMAF.loc[locusi,pop] #get pop af
                qjk = impMAF.loc[locusj,pop] #get pop af
                nk = popDict[pop]

                sums += (nk/sum(popDict.values()))*((qik*qjk)-(qi*qj))

            dijDict[locusi][locusj] = sums
        else:
            dijDict[locusi][locusj] = np.nan

filE = '/home/lindb/wbp/bayenv2/covariances/randmatrices/randDVALS/%s/%s_%s_randDVALS.txt'
if not op.exists('/home/lindb/wbp/bayenv2/covariances/randmatrices/randDVALS/'):
    os.makedirs('/home/lindb/wbp/bayenv2/covariances/randmatrices/randDVALS/')
DIR = op.dirname(filE)
if not op.exists(DIR):
    os.makedirs(DIR)

with open(filE,'w') as o:
    key0 = dijDict.keys()[0]
    line = '\\t'.join(dijDict[key0].keys()) + str('\\n')
    o.write("%%s" %% line)
    for locusi in dijDict.keys():
        line = str(locusi)+'\\t'+'\\t'.join([str(x) for x in dijDict[locusi].values()]) + str('\\n')
        o.write("%%s" %% line)


''' % (env,env,str(i).zfill(3),
       env,env,str(i).zfill(3))
        filE = '/home/lindb/wbp/bayenv2/covariances/randmatrices/randpyfiles/get_rand_dvals_%s_%s.py' % (env,
                                                                                                      str(i).zfill(3)
                                                                                                     )
        if not op.exists(op.dirname(filE)):
            os.makedirs(op.dirname(filE))
        with open(filE,'w') as o:
            o.write("%s" % text)

In [0]:
DIR = '/home/lindb/wbp/bayenv2/covariances/randmatrices/randpyfiles/'
pyfiles = [op.join(DIR,py) for py in ls(DIR)]
len(pyfiles)

In [0]:
#spread out the jobs across 25 files
#18000/200=90
count = 0
fcount = 0
for f in sorted(pyfiles):
    if count == 0:
        text = '''#!/bin/bash
#$ -N pyfile%s
#$ -V
#$ -j y
#$ -cwd

python %s\n''' % (str(fcount),
                  f)
    else:
        newtext = 'python %s\n' % f
        text = text + newtext
    count += 1
    if count == 90:
        count = 0
        fcount += 1
        filE = '/home/lindb/wbp/bayenv2/covariances/randmatrices/runpyfiles/%s_cmds.sh' % str(fcount).zfill(3)
        if not op.exists('/home/lindb/wbp/bayenv2/covariances/randmatrices/runpyfiles/'):
            os.makedirs('/home/lindb/wbp/bayenv2/covariances/randmatrices/runpyfiles/')
        with open(filE,'w') as o:
            o.write("%s" % text)

In [0]:
#qsub the qsub files

In [0]:
#check on progress godzilla
DIR = '/home/lindb/wbp/bayenv2/covariances/randmatrices/randDVALS/'
ds = [op.join(DIR,d) for d in ls(DIR)]
count = 0
for DIR in ds:
    count = count + len(ls(DIR))
count

In [0]:
DF = pd.read_csv('/home/lindb/wbp/bayenv2/covariances/dvals/Ann-ppt_dvals.txt',
                 header=0,index_col=0,sep='\t')
len(DF.index)

# get observed dvals
dvals = OrderedDict()
medvals = OrderedDict()
for env in get:
    print env
    DF = pd.read_csv('/home/lindb/wbp/bayenv2/covariances/dvals/%s_dvals.txt' % env,
                     header=0,index_col=0,sep="\t")
    dvals[env] = []
    for i,row in enumerate(DF.index):
        for j,col in enumerate(DF.columns):
            if i > j:
                dvals[env].append(abs(DF.loc[row,col]))

    DIR = '/home/lindb/wbp/bayenv2/covariances/randmatrices/randDVALS/%s/' % env
    files = [op.join(DIR,f) for f in ls(DIR)]

    fcount = 0
    medvals[env] = []
    for f in files:
        df = pd.read_csv(f,header=0,index_col=0,sep="\t")
        rvals = [] #random dij values
        for lst in df.values.tolist():
            for x in lst:
                if math.isnan(x) == False:
                    rvals.append(x)


        medvals[env].append(np.median([abs(val) for val in rvals]))

        fcount += 1
        if fcount % 100 == 0:
            print fcount

    filE = '/home/lindb/wbp/bayenv2/covariances/randmatrices/randmedvals/%s_randmedvalues.txt' % env
    if not op.exists(op.dirname(filE)):
        os.makedirs(op.dirname(filE))
    medvaldf = pd.DataFrame(medvals[env])
    medvaldf.to_csv(filE,header=False,index=False,sep="\t")

In [0]:
n5th = OrderedDict()    #get 95th percentile
gt_hth = OrderedDict()  #how much greater is the empirical median dvalue than the 100th percentile of random SNPs?
gt_n5th = OrderedDict() #how much greater is the empirical median dvalue than the 95th percentile of random SNPs?
bmax=OrderedDict()      #below what percentile of the observed dvals are the values less than the maximum randdvals
bn5th = OrderedDict()   #below what percentile of the observed dvals are the values less than the 95th perc randdvals
for env in sorted(get):
    print env,'has',len(intset[env]),'associated SNPs'
    sorts        = sorted(medvals[env])
    n5th[env]    = sorts[949]
    gt_hth[env]  = np.median(dvals[env])/max(medvals[env])
    print 'emp median is %sx greater than 100th percentile of random'%str(round(gt_hth[env],3))
    gt_n5th[env] = np.median(dvals[env])/n5th[env]
    print 'emp median is %sx greater than 95th percentile of random'%str(round(gt_n5th[env],3))
    for i,dval in enumerate(sorted(dvals[env])):
        if not dval < max(sorts): #the max(randDval) corresponds to the xth percentile of empirical d-values
            bmax[env]=(i+1,(i+1)/len(dvals[env]))
            break
    for i,dval in enumerate(sorted(dvals[env])):
        if not dval < sorts[949]:
            bn5th[env]=(i+1,(i+1)/len(dvals[env]))
            break
    print 'the max random value corresponds to the %s precentile' % str(round(bmax[env][1],3)*100),"(%s/%s)" % (bmax[env][0],len(dvals[env])),'of emp values'
    print 'the 95th perc rand val corresponds to the %s percentile',str(round(bn5th[env][1],3)*100),"(%s/%s)" % (bn5th[env][0],len(dvals[env])),'of emp values'
    print 'emp med dval =',"{:.4}".format(np.median(dvals[env]))
    print 'max rand dval =',"{:.4}".format(max(medvals[env]))
    print '95th rand dval =',"{:.4}".format(n5th[env])
    print '\n' 

In [0]:
#this is old- from when I calculated Dij using the maf of either allele within pops
n5th = OrderedDict()    #get 95th percentile
gt_hth = OrderedDict()  #how much greater is the empirical median dvalue than the 100th percentile of random SNPs?
gt_n5th = OrderedDict() #how much greater is the empirical median dvalue than the 95th percentile of random SNPs?
bmax=OrderedDict()      #below what percentile of the observed dvals are the values less than the maximum randdvals
bn5th = OrderedDict()   #below what percentile of the observed dvals are the values less than the 95th perc randdvals
for env in outlierdata.keys():
    print env,len(intset[env])
    sorts        = sorted(medvals[env])
    n5th[env]    = sorts[949]
    gt_hth[env]  = np.median(dvals[env])/max(medvals[env])
    print 'gt_hth',round(gt_hth[env],3)
    gt_n5th[env] = np.median(dvals[env])/n5th[env]
    print 'gt_n5th',round(gt_n5th[env],3)
    for i,dval in enumerate(sorted(dvals[env])):
        if not dval < max(sorts): #the max(randDval) corresponds to the xth percentile of empirical d-values
            bmax[env]=(i+1,i/len(dvals[env]))
            break
    for i,dval in enumerate(sorted(dvals[env])):
        if not dval < sorts[949]:
            bn5th[env]=(i+1,i/len(dvals[env]))
            break
    print 'bmax',round(bmax[env][1],3),"(%s/%s)" % (bmax[env][0],len(dvals[env]))
    print 'bn5th',round(bn5th[env][1],3),"(%s/%s)" % (bn5th[env][0],len(dvals[env]))
    print 'med dval',"{:.2}".format(np.median(dvals[env]))
    print 'max rand dval',"{:.4}".format(max(medvals[env]))
    print '\n'

In [0]:
df = pd.read_csv('/home/lindb/wbp/OutFLANK/imputed_MAF.txt',header=0,index_col=0,sep='\t')
df.head()

In [0]:
for pop in df:
    print pop,max(df[pop])

In [0]:
gtmin = 1000000000

gtmax = 0
for env in gt_hth.keys():
    if gt_hth[env] < gtmin:
        gtmin = gt_hth[env]
        minenv = env
    if gt_hth[env] > gtmax:
        gtmax = gt_hth[env]
        maxenv = env
print minenv,gtmin
print maxenv,gtmax

In [0]:
len(dvals[env]),len(dvals['Ann-ppt'])

# allele frequency shifts

In [0]:
for env in intset.keys():
    print env,len(intset[env])

In [0]:
#dataframe to get population MAF across 8 pops using all n=244 samples (can't use this for GEMMA since pop sizes r diff
impMAF.head()

In [0]:
for pop in impMAF:
    print pop,max(impMAF[pop])

In [0]:
#population sizes
popDict

In [0]:
rc = Client(profile="xmn")  
dview = rc[:]
lview = rc.load_balanced_view()
len(lview)

In [0]:
#do pairwise pops for empirical SNPs IDed by bayenv2 and calculate median Dij for each pop pair
def getshiftDict(tokens):
    import pandas as pd
    import numpy as np
    import os
    from collections import OrderedDict
    
    env,impMAF,popDict = tokens
    
    shiftDict = OrderedDict() 
    kcount = 0
    print env
    filE = '/home/lindb/wbp/bayenv2/results/sigsnps/%s_top_p05%%BF_1%%rho.txt' % env
    df = pd.read_csv(filE,header=0,sep='\t')
    outliersnps = df[env].tolist()
    for m,popm in enumerate(impMAF.columns):
        print popm
        shiftDict[popm] = OrderedDict()
        for l,popl in enumerate(impMAF.columns):
            if m>l: #only need to do the lower triangle - can reflect across diagonal later more quickly
                dijlist = []
                for i,locusi in enumerate(outliersnps):
                    for j,locusj in enumerate(outliersnps):
                        if i > j: #i=row, j=col : lower triangle 
                            sums =0 #Dij
                            kcount += 1
                            for popk in [popm,popl]:
                                qik = impMAF.loc[locusi,popk]        #get locusi maf for pop k
                                qjk = impMAF.loc[locusj,popk]        #get locusj maf for pop k
                                nk = popDict[popk]                   #N  individuals  in pop k

                                globN = 2*(popDict[popm]+popDict[popl]) # number of alleles across 2 pops

                                #get global mafs
                                fqim = impMAF.loc[locusi,popm]        #allele locusi freq  in popm
                                nqim = round(fqim*2*popDict[popm])    #allele locusi count in popm
                                fqil = impMAF.loc[locusi,popl]        #allele locusi freq  in popl
                                nqil = round(fqil*2*popDict[popl])    #allele locusi count in popl

                                fqjm = impMAF.loc[locusj,popm]        #allele locusj freq  in popm
                                nqjm = round(fqjm*2*popDict[popm])    #allele locusj count in popm
                                fqjl = impMAF.loc[locusj,popl]        #allele locusj freq  in popl
                                nqjl = round(fqjl*2*popDict[popl])    #allele locusj count in popl

                                qi = float(nqim+nqil)/float(globN) #global af locusi
                                qj = float(nqjm+nqjl)/float(globN) #global af locusj

                                sums += (float(nk)/float(popDict[popm]+popDict[popl]))*((qik*qjk)-(qi*qj))

                            dijlist.append(sums) #each pairwise pop comparison has a matrix of Dij
                        else:
                            pass #no redundancies, no diagonal. 
                shiftDict[popm][popl] = np.median([abs(d) for d in dijlist])
            else:
                shiftDict[popm][popl] = np.nan #no redundancies,no diagonal. will be faster to reflect across diag later
    #    if kcount > 1:
    #        break
    filE = '/home/lindb/wbp/bayenv2/freqshifts/%s_pop_pairwise_median_dij.txt' % env 
    with open(filE,'w') as o:
        key0 = shiftDict.keys()[0]
        line = '\t'.join(shiftDict[key0].keys()) + str('\n')
        o.write("%s" % line)
        for popk in shiftDict.keys():
            text = str(popk)+'\t'+'\t'.join([str(d) for d in shiftDict[popk].values()])+'\n'
            o.write("%s" % text)
    return "done"

In [0]:
len(files)

In [0]:
jobs = []
for env in sorted(intset):
    jobs.append(lview.apply_async(getshiftDict,[env,impMAF,popDict]))

In [0]:
count =0
for j in jobs:
    if j.ready():
        count +=1
count

In [0]:
#get pDij matrices, make them symmetric
shiftDF = OrderedDict()
for env in intset:
    filE = '/home/lindb/wbp/bayenv2/freqshifts/%s_pop_pairwise_median_dij.txt' % env
    shiftDF[env] = pd.read_csv(filE,header=0,index_col=0,sep='\t')
    for i,row in enumerate(shiftDF[env].index):
        for j,col in enumerate(shiftDF[env].columns):
            if i == j:
                shiftDF[env].loc[row,col] = 0
            elif math.isnan(shiftDF[env].loc[row,col]) == True:
                shiftDF[env].loc[row,col] = shiftDF[env].loc[col,row]
    filE = filE.split(".")[0]
    filE =  "%s_symm.txt" % filE
    print filE
    shiftDF[env].to_csv(filE,header=True,index=True,sep='\t')

In [0]:
shiftDF[env]

In [0]:
filE = '/home/lindb/wbp/bayenv2/distance_matrices/geographic_distances.txt'
geodist = pd.read_csv(filE,header=0,index_col=0,sep='\t')
geodist

In [0]:
#get rid of the km
for i,popi in enumerate(geodist.index):
    for j,popj in enumerate(geodist.columns):
        if i>j:
            geodist.loc[popi,popj] = float(geodist.loc[popi,popj].split()[0])
geodist

In [0]:
for i,popi in enumerate(geodist.index):
    for j,popj in enumerate(geodist.columns):
        if i == j:
            geodist.loc[popi,popj] = 0
        elif math.isnan(geodist.loc[popi,popj]) == True:
            geodist.loc[popi,popj] = geodist.loc[popj,popi]
geodist

In [0]:
#are pairwise population LD distances correlated to pairwise population geographic distances?
#corrected for pop freq
for env in sorted(shiftDF.keys()):
    print env, mantel(shiftDF[env],geodist,permutations=9999)

In [0]:
#are pairwise population LD distances correlated to pairwise population geographic distances?
#old
for env in shiftDF.keys():
    print env, mantel(shiftDF[env],geodist,permutations=9999)

In [0]:
#get environmental distance data
#files made in 09_OutFLANK
envdDict = OrderedDict()
for env in shiftDF.keys():
    filE = '/home/lindb/wbp/distance_matrices/%s_dist_symm.txt' % env
    envdDict[env] = pd.read_csv(filE,header=0,index_col=0,sep='\t')

In [0]:
#get data for scatter plots
shiftlst = OrderedDict()
for env in shiftDF.keys():
    shiftlst[env] = []
    for i,row in enumerate(shiftDF[env].index):
        for j,col in enumerate(shiftDF[env].columns):
            if i >j:
                shiftlst[env].append(shiftDF[env].loc[row,col])

In [0]:
with PdfPages('/home/lindb/wbp/figures/abs_dij_v_envdist_sameenv_AWSO-Lat.pdf') as pdf:
    plt.close('all')
    fig, ((a1,a2,a3),(a4,a5,a6),(a7,a8,a9)) = plt.subplots(3, 3, figsize=(5,5),dpi=400)

    a1 = plt.subplot(331)
    a1.scatter(shiftlst['AWS0-25'],distlst['AWS0-25'])
    a1.set_title('A',y=.9,loc='right',fontsize=20,fontweight='bold')
    a1.spines['right'].set_visible(False)
    a1.spines['top'].set_visible(False)
    a1.yaxis.set_ticks_position('left')
    a1.xaxis.set_ticks_position('bottom')
    
    a2 = plt.subplot(332)
    a2.scatter(shiftlst['AWS0-50'],distlst['AWS0-50'])
    a2.set_title('B',y=.9,loc='right',fontsize=20,fontweight='bold')
    a2.spines['right'].set_visible(False)
    a2.spines['top'].set_visible(False)
    a2.yaxis.set_ticks_position('left')
    a2.xaxis.set_ticks_position('bottom')
    
    a3 = plt.subplot(333)
    a3.scatter(shiftlst['Ann-ppt'],distlst['Ann-ppt'])
    a3.set_title('C',y=.9,loc='right',fontsize=20,fontweight='bold')
    a3.spines['right'].set_visible(False)
    a3.spines['top'].set_visible(False)
    a3.yaxis.set_ticks_position('left')
    a3.xaxis.set_ticks_position('bottom')
    
    a4 = plt.subplot(334)
    a4.scatter(shiftlst['CEC'],distlst['CEC'])
    a4.set_title('D',y=.9,loc='right',fontsize=20,fontweight='bold')
    a4.spines['right'].set_visible(False)
    a4.spines['top'].set_visible(False)
    a4.yaxis.set_ticks_position('left')
    a4.xaxis.set_ticks_position('bottom')
    
    a5 = plt.subplot(335)
    a5.scatter(shiftlst['Clay'],distlst['Clay'])
    a5.set_title('E',y=.9,loc='right',fontsize=20,fontweight='bold')
    a5.spines['right'].set_visible(False)
    a5.spines['top'].set_visible(False)
    a5.yaxis.set_ticks_position('left')
    a5.xaxis.set_ticks_position('bottom')
    
    a6 = plt.subplot(336)
    a6.scatter(shiftlst['Elev'],distlst['Elev'])
    a6.set_title('F',y=.9,loc='right',fontsize=20,fontweight='bold')
    a6.spines['right'].set_visible(False)
    a6.spines['top'].set_visible(False)
    a6.yaxis.set_ticks_position('left')
    a6.xaxis.set_ticks_position('bottom')
    
    a7 = plt.subplot(337)
    a7.scatter(shiftlst['GDD-Aug'],distlst['GDD-Aug'])
    a7.set_title('G',y=.9,loc='right',fontsize=20,fontweight='bold')
    a7.spines['right'].set_visible(False)
    a7.spines['top'].set_visible(False)
    a7.yaxis.set_ticks_position('left')
    a7.xaxis.set_ticks_position('bottom')
    
    a8 = plt.subplot(338)
    a8.scatter(shiftlst['GDD-May'],distlst['GDD-May'])
    a8.set_title('H',y=.9,loc='right',fontsize=20,fontweight='bold')
    a8.spines['right'].set_visible(False)
    a8.spines['top'].set_visible(False)
    a8.yaxis.set_ticks_position('left')
    a8.xaxis.set_ticks_position('bottom')
    
    a9 = plt.subplot(339)
    a9.scatter(shiftlst['Lat'],distlst['Lat'])
    a9.set_title('I',y=.9,loc='right',fontsize=20,fontweight='bold')
    a9.spines['right'].set_visible(False)
    a9.spines['top'].set_visible(False)
    a9.yaxis.set_ticks_position('left')
    a9.xaxis.set_ticks_position('bottom')
    
    fig.set_size_inches(13,10)
    pdf.savefig(fig,bbox_inches='tight')

In [0]:
with PdfPages('/home/lindb/wbp/figures/log_abs_dij_v_envdist_sameenv_AWSO-Lat.pdf') as pdf:
    plt.close('all')
    fig, ((a1,a2,a3),(a4,a5,a6),(a7,a8,a9)) = plt.subplots(3, 3, figsize=(5,5),dpi=400)

    a1 = plt.subplot(331)
    a1.scatter([math.log(x,10) for x in shiftlst['AWS0-25'] if x != 0],[x for i,x in enumerate(distlst['AWS0-25']) if shiftlst['AWS0-25'][i] !=0])
    a1.set_title('A',y=.9,loc='right',fontsize=20,fontweight='bold')
    a1.spines['right'].set_visible(False)
    a1.spines['top'].set_visible(False)
    a1.yaxis.set_ticks_position('left')
    a1.xaxis.set_ticks_position('bottom')
    
    a2 = plt.subplot(332)
    a2.scatter([math.log(x,10) for x in shiftlst['AWS0-50'] if x != 0],[x for i,x in enumerate(distlst['AWS0-50']) if shiftlst['AWS0-50'][i] !=0])
    a2.set_title('B',y=.9,loc='right',fontsize=20,fontweight='bold')
    a2.spines['right'].set_visible(False)
    a2.spines['top'].set_visible(False)
    a2.yaxis.set_ticks_position('left')
    a2.xaxis.set_ticks_position('bottom')
    
    a3 = plt.subplot(333)
    a3.scatter([math.log(x,10) for x in shiftlst['Ann-ppt'] if x != 0],[x for i,x in enumerate(distlst['Ann-ppt']) if shiftlst['Ann-ppt'][i] !=0])
    a3.set_title('C*',y=.9,loc='right',fontsize=20,fontweight='bold')
    a3.spines['right'].set_visible(False)
    a3.spines['top'].set_visible(False)
    a3.yaxis.set_ticks_position('left')
    a3.xaxis.set_ticks_position('bottom')
    
    a4 = plt.subplot(334)
    a4.scatter([math.log(x,10) for x in shiftlst['CEC'] if x != 0],[x for i,x in enumerate(distlst['CEC']) if shiftlst['CEC'][i] !=0])
    a4.set_title('D',y=.9,loc='right',fontsize=20,fontweight='bold')
    a4.spines['right'].set_visible(False)
    a4.spines['top'].set_visible(False)
    a4.yaxis.set_ticks_position('left')
    a4.xaxis.set_ticks_position('bottom')
    
    a5 = plt.subplot(335)
    a5.scatter([math.log(x,10) for x in shiftlst['Clay'] if x != 0],[x for i,x in enumerate(distlst['Clay']) if shiftlst['Clay'][i] !=0])
    a5.set_title('E',y=.9,loc='right',fontsize=20,fontweight='bold')
    a5.spines['right'].set_visible(False)
    a5.spines['top'].set_visible(False)
    a5.yaxis.set_ticks_position('left')
    a5.xaxis.set_ticks_position('bottom')
    
    a6 = plt.subplot(336)
    a6.scatter([math.log(x,10) for x in shiftlst['Elev'] if x != 0],[x for i,x in enumerate(distlst['Elev']) if shiftlst['Elev'][i] !=0])
    a6.set_title('F',y=.9,loc='right',fontsize=20,fontweight='bold')
    a6.spines['right'].set_visible(False)
    a6.spines['top'].set_visible(False)
    a6.yaxis.set_ticks_position('left')
    a6.xaxis.set_ticks_position('bottom')
    
    a7 = plt.subplot(337)
    a7.scatter([math.log(x,10) for x in shiftlst['GDD-Aug'] if x != 0],[x for i,x in enumerate(distlst['GDD-Aug']) if shiftlst['GDD-Aug'][i] !=0])
    a7.set_title('G',y=.9,loc='right',fontsize=20,fontweight='bold')
    a7.spines['right'].set_visible(False)
    a7.spines['top'].set_visible(False)
    a7.yaxis.set_ticks_position('left')
    a7.xaxis.set_ticks_position('bottom')
    
    a8 = plt.subplot(338)
    a8.scatter([math.log(x,10) for x in shiftlst['GDD-May'] if x != 0],[x for i,x in enumerate(distlst['GDD-May']) if shiftlst['GDD-May'][i] !=0])
    a8.set_title('H*',y=.9,loc='right',fontsize=20,fontweight='bold')
    a8.spines['right'].set_visible(False)
    a8.spines['top'].set_visible(False)
    a8.yaxis.set_ticks_position('left')
    a8.xaxis.set_ticks_position('bottom')
    
    a9 = plt.subplot(339)
    a9.scatter([math.log(x,10) for x in shiftlst['Lat'] if x != 0],[x for i,x in enumerate(distlst['Lat']) if shiftlst['Lat'][i] !=0])
    a9.set_title('I',y=.9,loc='right',fontsize=20,fontweight='bold')
    a9.spines['right'].set_visible(False)
    a9.spines['top'].set_visible(False)
    a9.yaxis.set_ticks_position('left')
    a9.xaxis.set_ticks_position('bottom')
    
    fig.set_size_inches(13,10)
    pdf.savefig(fig,bbox_inches='tight')

In [0]:
#distance correlations for env Dij among outliers against the same env Euclidian distance 
#corrected
for env in sorted(shiftDF.keys()):
    mant = mantel(shiftDF[env],envdDict[env],permutations=9999)
    if mant[1] <= 0.05:
        print env, mant

In [0]:
#distance correlations for env Dij among outliers against the same env Euclidian distance 
#old
for env in shiftDF.keys():
    mant = mantel(shiftDF[env],envdDict[env],permutations=9999)
    if mant[1] <= 0.05:
        print env, mant

In [0]:
#distance correlations for env Dij among outliers against other env Euclidian distances - deleted print to comment
#corrected
mant = OrderedDict()
for i,envi in enumerate(sorted(shiftDF.keys())):
    mant[envi] = OrderedDict()
    for j,envj in enumerate(sorted(envdDict.keys())):
        if i > j:
            mant[envi][envj] = mantel(shiftDF[envi],envdDict[envj],permutations=9999) # ,permutations=9999
            #print envi,envj, mant[1]
            if mant[envi][envj][1] <= 0.05:
                print envi,'loci Dij vs ',envj,'Euclidian distances\n',mant[envi][envj],'\n' 

In [0]:
#distance correlations for env Dij among outliers against other env Euclidian distances - deleted print to comment
#old
mant = OrderedDict()
for i,envi in enumerate(shiftDF.keys()):
    mant[envi] = OrderedDict()
    for j,envj in enumerate(envdDict.keys()):
        if i > j:
            mant[envi][envj] = mantel(shiftDF[envi],envdDict[envj],permutations=9999) # ,permutations=9999
            #print envi,envj, mant[1]
            if mant[envi][envj][1] <= 0.05:
                print envi,'loci Dij vs ',envj,'Euclidian distances\n',mant[envi][envj],'\n' 
#Lon loci Dij vs  Ann-ppt Euclidian distances
#(0.7716010350739726, 0.0016000000000000001, 8) 
#
#Lon loci Dij vs  Lat Euclidian distances
#(-0.42566652606664973, 0.0332, 8) 
#
#Rock-cov loci Dij vs  Ann-ppt Euclidian distances
#(0.55424357362000032, 0.022200000000000001, 8) 
#
#Rock-cov loci Dij vs  Lon Euclidian distances
#(0.55685544364774331, 0.0252, 8) 
#
#Sand loci Dij vs  Clay Euclidian distances
#(0.53454002690746905, 0.0207, 8) 
#
#Silt loci Dij vs  Sand Euclidian distances
#(0.44079182582652993, 0.023699999999999999, 8) 
#
#Tmin-Jan loci Dij vs  Ann-ppt Euclidian distances
#(0.57650137127397227, 0.0135, 8) 
#
#Tmin-Jan loci Dij vs  Lon Euclidian distances
#(0.48221840890815115, 0.028799999999999999, 8) 
#
#WC15Bar loci Dij vs  Tmax-July Euclidian distances
#(0.34904867738608325, 0.028500000000000001, 8) 
#
#WC3rdbar loci Dij vs  AWS0-25 Euclidian distances
#(0.43294142988706169, 0.036600000000000001, 8) 
#
#WC3rdbar loci Dij vs  AWS0-50 Euclidian distances
#(0.45379228695092727, 0.046699999999999998, 8) 
#
#WC3rdbar loci Dij vs  Tmax-July Euclidian distances
#(0.45389697231068271, 0.0030999999999999999, 8) 
#
#WC3rdbar loci Dij vs  WC15Bar Euclidian distances
#(0.51257777427440188, 0.029499999999999998, 8) 

In [0]:
len(sigenvs)

In [0]:
#look for patters from euclidian dists
#corrected
eucs = OrderedDict()
for i,envi in enumerate(sorted(envdDict.keys())):
    eucs[envi] = OrderedDict()
    for j,envj in enumerate(sorted(envdDict.keys())):
        if i > j:
            eucs[envi][envj] = mantel(envdDict[envi],envdDict[envj],permutations = 9999)
            if eucs[envi][envj][1] <=0.05:
                print envi,envj,eucs[envi][envj]

In [0]:
#look for patters from euclidian dists
#old
eucs = OrderedDict()
for i,envi in enumerate(envdDict.keys()):
    eucs[envi] = OrderedDict()
    for j,envj in enumerate(envdDict.keys()):
        if i > j:
            eucs[envi][envj] = mantel(envdDict[envi],envdDict[envj],permutations = 9999)
            if eucs[envi][envj][1] <=0.05:
                print envi,envj,eucs[envi][envj]
#AWS0-50 AWS0-25 (0.92563195780663798, 0.00059999999999999995, 8)
#Clay CEC (0.94243026430775967, 0.0015, 8)
#Lat Elev (0.39876994024295709, 0.045199999999999997, 8)
#Lon Ann-ppt (0.7145343340679835, 0.0022000000000000001, 8)
#Max-rad-input Lat (0.46285891784178229, 0.033000000000000002, 8)
#Sand AWS0-50 (0.43931703114531706, 0.018800000000000001, 8)
#Sand Clay (0.3723300732412157, 0.032500000000000001, 8)
#Silt AWS0-25 (0.56910810657784439, 0.020500000000000001, 8)
#Silt AWS0-50 (0.75523793270279638, 0.0057000000000000002, 8)
#Silt Sand (0.83954421938148982, 0.0001, 8)
#Tmin-Jan GDD-Aug (0.45450267193847571, 0.0361, 8)
#WC15Bar AWS0-25 (0.67219012746044704, 0.01, 8)
#WC15Bar AWS0-50 (0.8511126463087334, 0.002, 8)
#WC15Bar Silt (0.68065119836969423, 0.0152, 8)
#WC3rdbar AWS0-25 (0.60929292004599522, 0.0177, 8)
#WC3rdbar AWS0-50 (0.77608813807768406, 0.0057000000000000002, 8)
#WC3rdbar Silt (0.55818888588782412, 0.030099999999999998, 8)
#WC3rdbar WC15Bar (0.94230567933893883, 0.00020000000000000001, 8)

In [0]:
#patterns of environmental distance to geographic distance
#corrected
for env in sorted(envdDict.keys()):
    mant = mantel(envdDict[env],geodist,permutations=9999)
    #if mant[1] <= 0.05:
        #print env,mant
    print env,mant

In [0]:
#patterns of environmental distance to geographic distance
#old
for env in envdDict.keys():
    mant = mantel(envdDict[env],geodist,permutations=9999)
    #if mant[1] <= 0.05:
        #print env,mant
    print env,mant
#AWS0-25 (-0.10608611710064117, 0.6159, 8)
#AWS0-50 (-0.10662414876775597, 0.69989999999999997, 8)
#Ann-ppt (-0.1825290530704001, 0.28789999999999999, 8)
#CEC (0.24308086638413512, 0.15240000000000001, 8)
#Clay (0.27161756617669391, 0.091499999999999998, 8)
#Elev (0.40617770554536675, 0.044400000000000002, 8)
#GDD-Aug (0.060192865042875195, 0.73899999999999999, 8)
#GDD-May (0.091559881387966044, 0.74639999999999995, 8)
#Lat (0.96306650501124502, 0.00020000000000000001, 8)
#Lon (0.11594248224889436, 0.51459999999999995, 8)
#Max-rad-input (0.39916883319045005, 0.044600000000000001, 8)
#Rock-cov (-0.0013653758034187704, 0.99319999999999997, 8)
#Sand (-0.065400320530522443, 0.70289999999999997, 8)
#Silt (-0.11938536549339876, 0.5847, 8)
#Tmax-July (0.36685027020170108, 0.0717, 8)
#Tmin-Jan (-0.008502374494668636, 0.96970000000000001, 8)
#WC15Bar (0.080797777802775472, 0.79679999999999995, 8)
#WC3rdbar (0.19001363716445102, 0.30890000000000001, 8)

# split positive and negative Dij and run mantel tests

In [0]:
#do pairwise pops for empirical SNPs IDed by bayenv2 and calculate median Dij for each pop pair
def getrawshiftDict(tokens):
    import pandas as pd
    import numpy as np
    import os
    from collections import OrderedDict
    
    env,impMAF,popDict = tokens
    
    shiftDictpos = OrderedDict() 
    shiftDictneg = OrderedDict()
    kcount = 0
    print env
    filE = '/home/lindb/wbp/bayenv2/results/sigsnps/%s_top_p05%%BF_1%%rho.txt' % env
    df = pd.read_csv(filE,header=0,sep='\t')
    outliersnps = df[env].tolist()
    for m,popm in enumerate(impMAF.columns):
        print popm
        shiftDictpos[popm] = OrderedDict()
        shiftDictneg[popm] = OrderedDict()
        for l,popl in enumerate(impMAF.columns):
            if m>l: #only need to do the lower triangle - can reflect across diagonal later more quickly
                dijlist = []
                for i,locusi in enumerate(outliersnps):
                    for j,locusj in enumerate(outliersnps):
                        if i > j: #i=row, j=col : lower triangle 
                            sums =0 #Dij
                            kcount += 1
                            for popk in [popm,popl]:
                                qik = impMAF.loc[locusi,popk]        #get locusi maf for pop k
                                qjk = impMAF.loc[locusj,popk]        #get locusj maf for pop k
                                nk = popDict[popk]                   #N  individuals  in pop k

                                globN = 2*(popDict[popm]+popDict[popl]) # number of alleles across 2 pops

                                #get global mafs
                                fqim = impMAF.loc[locusi,popm]        #minor allele locusi freq  in popm
                                nqim = round(fqim*2*popDict[popm])    #minor allele locusi count in popm
                                fqil = impMAF.loc[locusi,popl]        #minor allele locusi freq  in popl
                                nqil = round(fqil*2*popDict[popl])    #minor allele locusi count in popl

                                fqjm = impMAF.loc[locusj,popm]        #minor allele locusj freq  in popm
                                nqjm = round(fqjm*2*popDict[popm])    #minor allele locusj count in popm
                                fqjl = impMAF.loc[locusj,popl]        #minor allele locusj freq  in popl
                                nqjl = round(fqjl*2*popDict[popl])    #minor allele locusj count in popl

                                qi = float(nqim+nqil)/float((2*popDict[popm])+(2*popDict[popl])) #global maf locusi
                                qj = float(nqjm+nqjl)/float((2*popDict[popm])+(2*popDict[popl])) #global maf locusj

                                sums += (float(nk)/float(popDict[popm]+popDict[popl]))*((qik*qjk)-(qi*qj))

                            dijlist.append(sums) #each pairwise pop comparison has a matrix of Dij
                        else:
                            pass #no redundancies, no diagonal. 
                pos = [x for x in dijlist if x >= 0]
                neg = [x for x in dijlist if x <=  0]
                shiftDictpos[popm][popl] = np.median([d for d in pos])
                shiftDictneg[popm][popl] = np.median([d for d in neg])
                
            else:
                shiftDictpos[popm][popl] = np.nan #no redundancies,no diagonal. will be faster to reflect across diag later
                shiftDictneg[popm][popl] = np.nan #no redundancies,no diagonal. will be faster to reflect across diag later
    #    if kcount > 1:
    #        break
    filE = '/home/lindb/wbp/bayenv2/freqshifts/%s_pop_pairwise_median_positive_dij_withzeros.txt' % env 
    with open(filE,'w') as o:
        key0 = shiftDictpos.keys()[0]
        line = '\t'.join(shiftDictpos[key0].keys()) + str('\n')
        o.write("%s" % line)
        for popk in shiftDictpos.keys():
            text = str(popk)+'\t'+'\t'.join([str(d) for d in shiftDictpos[popk].values()])+'\n'
            o.write("%s" % text)
            
    filE = '/home/lindb/wbp/bayenv2/freqshifts/%s_pop_pairwise_median_negative_dij_withzeros.txt' % env 
    with open(filE,'w') as o:
        key0 = shiftDictneg.keys()[0]
        line = '\t'.join(shiftDictneg[key0].keys()) + str('\n')
        o.write("%s" % line)
        for popk in shiftDictneg.keys():
            text = str(popk)+'\t'+'\t'.join([str(d) for d in shiftDictneg[popk].values()])+'\n'
            o.write("%s" % text)
    return "done"

In [0]:
jobs = []
for env in outlierdata.keys():
    jobs.append(lview.apply_async(getrawshiftDict,[env,impMAF,popDict]))

In [0]:
count = 0
for j in jobs:
    if j.ready():
        count += 1
count

In [0]:
nozeroposDF = OrderedDict()
for env in outlierdata.keys():
    filE = '/home/lindb/wbp/bayenv2/freqshifts/%s_pop_pairwise_median_positive_dij_nozeros.txt' % env
    nozeroposDF[env] = pd.read_csv(filE,header=0,index_col=0,sep='\t')
    for i,row in enumerate(nozeroposDF[env].index):
        for j,col in enumerate(nozeroposDF[env].columns):
            if i == j:
                nozeroposDF[env].loc[row,col] = 0
            elif math.isnan(float(nozeroposDF[env].loc[row,col])) == True:
                nozeroposDF[env].loc[row,col] = nozeroposDF[env].loc[col,row]
    filE = filE.split(".")[0]
    filE =  "%s_symm.txt" % filE
    print filE
    nozeroposDF[env].to_csv(filE,header=True,index=True,sep='\t')

In [0]:
nozeroposDF['Ann-ppt']

In [0]:
nozeronegDF = OrderedDict()
for env in outlierdata.keys():
    filE = '/home/lindb/wbp/bayenv2/freqshifts/%s_pop_pairwise_median_negative_dij_nozeros.txt' % env
    nozeronegDF[env] = pd.read_csv(filE,header=0,index_col=0,sep='\t')
    for i,row in enumerate(nozeronegDF[env].index):
        for j,col in enumerate(nozeronegDF[env].columns):
            if i == j:
                nozeronegDF[env].loc[row,col] = 0
            elif nozeronegDF[env].loc[row,col] == 'zero':
                print 'zero',nozeronegDF[env].loc[row,col]
                nozeronegDF[env].loc[row,col] = float(0)
                print 'changed',nozeronegDF[env].loc[row,col]
            elif math.isnan(float(nozeronegDF[env].loc[row,col])) == True:
                nozeronegDF[env].loc[row,col] = nozeronegDF[env].loc[col,row]
    filE = filE.split(".")[0]
    filE =  "%s_symm.txt" % filE
    print filE
    nozeronegDF[env].to_csv(filE,header=True,index=True,sep='\t')

In [0]:
nozeronegDF['Ann-ppt']

In [0]:
withzeroposDF = OrderedDict()
for env in outlierdata.keys():
    filE = '/home/lindb/wbp/bayenv2/freqshifts/%s_pop_pairwise_median_positive_dij_withzeros.txt' % env
    withzeroposDF[env] = pd.read_csv(filE,header=0,index_col=0,sep='\t')
    for i,row in enumerate(withzeroposDF[env].index):
        for j,col in enumerate(withzeroposDF[env].columns):
            if i == j:
                withzeroposDF[env].loc[row,col] = 0
            elif math.isnan(float(withzeroposDF[env].loc[row,col])) == True:
                withzeroposDF[env].loc[row,col] = withzeroposDF[env].loc[col,row]
    filE = filE.split(".")[0]
    filE =  "%s_symm.txt" % filE
    print filE
    withzeroposDF[env].to_csv(filE,header=True,index=True,sep='\t')

In [0]:
withzeronegDF = OrderedDict()
for env in outlierdata.keys():
    filE = '/home/lindb/wbp/bayenv2/freqshifts/%s_pop_pairwise_median_negative_dij_withzeros.txt' % env
    withzeronegDF[env] = pd.read_csv(filE,header=0,index_col=0,sep='\t')
    for i,row in enumerate(withzeronegDF[env].index):
        for j,col in enumerate(withzeronegDF[env].columns):
            if i == j:
                withzeronegDF[env].loc[row,col] = 0
            elif math.isnan(float(withzeronegDF[env].loc[row,col])) == True:
                withzeronegDF[env].loc[row,col] = withzeronegDF[env].loc[col,row]
    filE = filE.split(".")[0]
    filE =  "%s_symm.txt" % filE
    print filE
    withzeronegDF[env].to_csv(filE,header=True,index=True,sep='\t')

In [0]:
outlierdata.keys()

In [0]:
def setup_r(): 
    os.environ['R_HOME'] = '/home/lindb/g/R3/lib64/R/' 
    os.environ['LD_LIBRARY_PATH'] = "%s/lib:%s" % (os.environ['R_HOME'], 
                                                   os.environ['LD_LIBRARY_PATH'])

In [0]:
setup_r()
import readline
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri as pd2ri
pd2ri.activate()
r = robjects.r

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext rpy2.ipython
%matplotlib inline

In [0]:
%%R
sessionInfo()

In [0]:
%%R
envs = c('AWS0-25',
 'AWS0-50',
 'Ann-ppt',
 'CEC',
 'Clay',
 'Elev',
 'GDD-Aug',
 'GDD-May',
 'Lat',
 'Lon',
 'Max-rad-input',
 'Rock-cov',
 'Sand',
 'Silt',
 'Tmax-July',
 'Tmin-Jan',
 'WC15Bar',
 'WC3rdbar')

In [0]:
%%R
#read in the Dij files
library(stringr)
nozerosneg = list()
for (env in envs){
    i = which(envs == env)
    f = "/home/lindb/wbp/bayenv2/freqshifts/%s_pop_pairwise_median_negative_dij_nozeros_symm.txt"
    filE = str_replace(f,"%s",env)
    nozerosneg[[i]] = read.table(filE,sep='\t',header=TRUE,row.names=1)
    names(nozerosneg)[i] = env
}
wzerosneg = list()
for (env in envs){
    i = which(envs == env)
    f = "/home/lindb/wbp/bayenv2/freqshifts/%s_pop_pairwise_median_negative_dij_withzeros_symm.txt"
    filE = str_replace(f,"%s",env)
    wzerosneg[[i]] = read.table(filE,sep='\t',header=TRUE,row.names=1)
    names(wzerosneg)[i] = env
}
nozerospos = list()
for (env in envs){
    i = which(envs == env)
    f = "/home/lindb/wbp/bayenv2/freqshifts/%s_pop_pairwise_median_positive_dij_nozeros_symm.txt"
    filE = str_replace(f,"%s",env)
    nozerospos[[i]] = read.table(filE,sep='\t',header=TRUE,row.names=1)
    names(nozerospos)[i] = env
}
wzerospos = list()
for (env in envs){
    i = which(envs == env)
    f = "/home/lindb/wbp/bayenv2/freqshifts/%s_pop_pairwise_median_positive_dij_withzeros_symm.txt"
    filE = str_replace(f,"%s",env)
    wzerospos[[i]] = read.table(filE,sep='\t',header=TRUE,row.names=1)
    names(wzerospos)[i] = env
}

In [0]:
%%R
#get the environmental distance matrices
envdist = list()
for (env in envs){
    i = which(envs==env)
    f = "/home/lindb/wbp/distance_matrices/%s_dist_symm.txt"
    filE = str_replace(f,"%s",env)
    envdist[[i]] = read.table(filE,sep='\t',header=TRUE,row.names=1)
    names(envdist)[i] = env
}

In [0]:
%%R
library(vegan)

In [0]:
%%R
nozerosneg

In [0]:
%%R
#run mantel tests for neg Dijs calculated without zeros
for(env in envs){
    i = which(envs==env)
    mant = mantel(nozerosneg[[env]],envdist[[env]],method="pearson",permutations=999,na.rm=TRUE)
    if (mant$signif <= 0.05){
        print(env)
        print(mant)        
    }
}

In [0]:
%%R
#run mantel tests for neg Dijs calculated with zeros
for(env in envs){
    i = which(envs==env)
    mant = mantel(wzerosneg[[env]],envdist[[env]],method="pearson",permutations=999,na.rm=TRUE)
    if (mant$signif <= 0.05){
        print(env)
        print(mant)        
    }
}

In [0]:
%%R
#run mantel tests for pos Dijs calculated with zeros
for(env in envs){
    mant = mantel(wzerospos[[env]],envdist[[env]],method="pearson",permutations=999,na.rm=TRUE)
    if (mant$signif <= 0.05){
        print(env)
        print(mant)        
    }
}

In [0]:
%%R
#run mantel tests for pos Dijs calculated without zeros
for(env in envs){
    mant = mantel(nozerospos[[env]],envdist[[env]],method="pearson",permutations=999,na.rm=TRUE)
    if (mant$signif <= 0.05){
        print(env)
        print(mant)        
    }
}

In [0]:
%%R
annpptnozeros = nozerospos[['Ann-ppt']][lower.tri(nozerospos[['Ann-ppt']])]
annpptnozeros

In [0]:
%%R
apptenvdist = envdist[['Ann-ppt']][lower.tri(envdist[['Ann-ppt']])]
apptenvdist

In [0]:
%%R
plot(annpptnozeros,apptenvdist)

In [0]:
#distance correlations for env Dij among outliers against the same env Euclidian distance - deleted print to comment
for env in posshiftDF.keys():
    mant = mantel(posshiftDF[env],envdDict[env],permutations=9999)
    if mant[1] <= 0.05:
        print env, mant

In [0]:
#distance correlations for env Dij among outliers against other env Euclidian distances - deleted print to comment
posmant= OrderedDict()
for i,envi in enumerate(posshiftDF.keys()):
    posmant[envi] = OrderedDict()
    for j,envj in enumerate(posshiftDF.keys()):
        if i > j:
            posmant[envi][envj] = mantel(posshiftDF[envi],envdDict[envj],permutations=999) # ,permutations=9999
            #print envi,envj, posmant[1]
            if posmant[envi][envj][1] <= 0.05:
                print envi,'loci Dij vs ',envj,'Euclidian distances\n',posmant[envi][envj],'\n' 

In [0]:
#distance correlations for env Dij among outliers against the same env Euclidian distance - deleted print to comment
for env in negshiftDF.keys():
    mant = mantel(negshiftDF[env],envdDict[env],permutations=999)
    if mant[1] <= 0.05:
        print env, mant
#AWS0-50 (-0.5187120808294664, 0.048000000000000001, 8)
#Elev (-0.38386332885151592, 0.035999999999999997, 8)
#GDD-May (-0.49436024387658395, 0.025999999999999999, 8)
#Lat (-0.38351080073888355, 0.047, 8)
#Lon (-0.39247528366555717, 0.017000000000000001, 8)
#Silt (-0.50109539481141629, 0.035000000000000003, 8)
#WC15Bar (-0.55864761263082974, 0.050000000000000003, 8)

In [0]:
#distance correlations for env Dij among outliers against other env Euclidian distances - deleted print to comment
negmant= OrderedDict()
for i,envi in enumerate(negshiftDF.keys()):
    negmant[envi] = OrderedDict()
    for j,envj in enumerate(negshiftDF.keys()):
        if i > j:
            negmant[envi][envj] = mantel(negshiftDF[envi],envdDict[envj],permutations=999) # ,permutations=9999
            #print envi,envj, negmant[1]
            if negmant[envi][envj][1] <= 0.05:
                print envi,'loci Dij vs ',envj,'Euclidian distances\n',negmant[envi][envj],'\n' 
#AWS0-50 loci Dij vs  AWS0-25 Euclidian distances
#(-0.45711120411044837, 0.049000000000000002, 8) 
#
#CEC loci Dij vs  Ann-ppt Euclidian distances
#(-0.39739357529313168, 0.032000000000000001, 8) 
#
#GDD-Aug loci Dij vs  Elev Euclidian distances
#(-0.43586420866039266, 0.031, 8) 
#
#Lat loci Dij vs  CEC Euclidian distances
#(-0.54522328655821961, 0.033000000000000002, 8) 
#
#Lat loci Dij vs  Elev Euclidian distances
#(-0.42828652409270873, 0.032000000000000001, 8) 
#
#Lon loci Dij vs  Ann-ppt Euclidian distances
#(-0.44268477932420164, 0.017000000000000001, 8) 
#
#Max-rad-input loci Dij vs  Ann-ppt Euclidian distances
#(-0.49073697619324924, 0.014999999999999999, 8) 
#
#Max-rad-input loci Dij vs  Lon Euclidian distances
#(-0.36898951713648914, 0.045999999999999999, 8) 
#
#Sand loci Dij vs  Clay Euclidian distances
#(-0.50749772113017022, 0.021000000000000001, 8) 
#
#Silt loci Dij vs  AWS0-25 Euclidian distances
#(-0.45042297309726187, 0.047, 8) 
#
#Silt loci Dij vs  Sand Euclidian distances
#(-0.38456814005183487, 0.047, 8) 
#
#WC15Bar loci Dij vs  AWS0-25 Euclidian distances
#(-0.51462685560320931, 0.029000000000000001, 8) 
#
#WC15Bar loci Dij vs  AWS0-50 Euclidian distances
#(-0.5537241770139858, 0.037999999999999999, 8)

In [0]:
neglst = OrderedDict()
for env in envdDict.keys():
    neglst[env] = []
    for i,row in enumerate(negshiftDF[env].index):
        for j,col in enumerate(negshiftDF[env].columns):
            if i > j:
                neglst[env].append(negshiftDF[env].loc[row,col])

In [0]:
poslst = OrderedDict()
for env in envdDict.keys():
    poslst[env] = []
    for i,row in enumerate(posshiftDF[env].index):
        for j,col in enumerate(posshiftDF[env].columns):
            if i > j:
                poslst[env].append(posshiftDF[env].loc[row,col])

In [0]:
distlst = OrderedDict()
for env in envdDict.keys():
    distlst[env] = []
    for i,row in enumerate(envdDict[env].index):
        for j,col in enumerate(envdDict[env].columns):
            if i > j:
                distlst[env].append(envdDict[env].loc[row,col])

In [0]:
#with PdfPages('/home/lindb/wbp/figures/neg_dij_v_envdist_sameenv_AWSO-Lat.pdf') as pdf:
plt.close('all')
fig, ((a1,a2,a3),(a4,a5,a6),(a7,a8,a9)) = plt.subplots(3, 3, figsize=(5,5),dpi=400)

a1 = plt.subplot(331)
a1.scatter(neglst['AWS0-25'],distlst['AWS0-25'])
a1.set_title('AWS0-25',y=.9,loc='right',fontsize=20,fontweight='bold')
a1.spines['right'].set_visible(False)
a1.spines['top'].set_visible(False)
a1.yaxis.set_ticks_position('left')
a1.xaxis.set_ticks_position('bottom')

a2 = plt.subplot(332)
a2.scatter(neglst['AWS0-50'],distlst['AWS0-50'])
a2.set_title('AWS0-50',y=.9,loc='right',fontsize=20,fontweight='bold')
a2.spines['right'].set_visible(False)
a2.spines['top'].set_visible(False)
a2.yaxis.set_ticks_position('left')
a2.xaxis.set_ticks_position('bottom')

a3 = plt.subplot(333)
a3.scatter(neglst['Ann-ppt'],distlst['Ann-ppt'])
a3.set_title('Ann-ppt',y=.9,loc='right',fontsize=20,fontweight='bold')
a3.spines['right'].set_visible(False)
a3.spines['top'].set_visible(False)
a3.yaxis.set_ticks_position('left')
a3.xaxis.set_ticks_position('bottom')

a4 = plt.subplot(334)
a4.scatter(neglst['CEC'],distlst['CEC'])
a4.set_title('CEC',y=.9,loc='right',fontsize=20,fontweight='bold')
a4.spines['right'].set_visible(False)
a4.spines['top'].set_visible(False)
a4.yaxis.set_ticks_position('left')
a4.xaxis.set_ticks_position('bottom')

a5 = plt.subplot(335)
a5.scatter(neglst['Clay'],distlst['Clay'])
a5.set_title('Clay',y=.9,loc='right',fontsize=20,fontweight='bold')
a5.spines['right'].set_visible(False)
a5.spines['top'].set_visible(False)
a5.yaxis.set_ticks_position('left')
a5.xaxis.set_ticks_position('bottom')

a6 = plt.subplot(336)
a6.scatter(neglst['Elev'],distlst['Elev'])
a6.set_title('Elev',y=.9,loc='right',fontsize=20,fontweight='bold')
a6.spines['right'].set_visible(False)
a6.spines['top'].set_visible(False)
a6.yaxis.set_ticks_position('left')
a6.xaxis.set_ticks_position('bottom')

a7 = plt.subplot(337)
a7.scatter(neglst['GDD-Aug'],distlst['GDD-Aug'])
a7.set_title('GDD-Aug',y=.9,loc='right',fontsize=20,fontweight='bold')
a7.spines['right'].set_visible(False)
a7.spines['top'].set_visible(False)
a7.yaxis.set_ticks_position('left')
a7.xaxis.set_ticks_position('bottom')

a8 = plt.subplot(338)
a8.scatter(neglst['GDD-May'],distlst['GDD-May'])
a8.set_title('GDD-May',y=.9,loc='right',fontsize=20,fontweight='bold')
a8.spines['right'].set_visible(False)
a8.spines['top'].set_visible(False)
a8.yaxis.set_ticks_position('left')
a8.xaxis.set_ticks_position('bottom')

a9 = plt.subplot(339)
a9.scatter(neglst['Lat'],distlst['Lat'])
a9.set_title('Lat',y=.9,loc='right',fontsize=20,fontweight='bold')
a9.spines['right'].set_visible(False)
a9.spines['top'].set_visible(False)
a9.yaxis.set_ticks_position('left')
a9.xaxis.set_ticks_position('bottom')

fig.set_size_inches(13,10)
#pdf.savefig(fig,bbox_inches='tight')

In [0]:
#with PdfPages('/home/lindb/wbp/figures/log_neg_dij_v_envdist_sameenv_nozeros_AWSO-Lat.pdf') as pdf:
plt.close('all')
fig, ((a1,a2,a3),(a4,a5,a6),(a7,a8,a9)) = plt.subplots(3, 3, figsize=(5,5),dpi=400)

a1 = plt.subplot(331)
a1.scatter([x for x in neglst['AWS0-25'] if x != 0],[x for i,x in enumerate(distlst['AWS0-25']) if neglst['AWS0-25'][i] != 0])
a1.set_title('A',y=.9,loc='right',fontsize=20,fontweight='bold')
a1.spines['right'].set_visible(False)
a1.spines['top'].set_visible(False)
a1.yaxis.set_ticks_position('left')
a1.xaxis.set_ticks_position('bottom')

a2 = plt.subplot(332)
a2.scatter([x for x in neglst['AWS0-50'] if x != 0],[x for i,x in enumerate(distlst['AWS0-50']) if neglst['AWS0-50'][i] != 0])
a2.set_title('B',y=.9,loc='right',fontsize=20,fontweight='bold')
a2.spines['right'].set_visible(False)
a2.spines['top'].set_visible(False)
a2.yaxis.set_ticks_position('left')
a2.xaxis.set_ticks_position('bottom')

a3 = plt.subplot(333)
a3.scatter([x for x in neglst['Ann-ppt'] if x != 0],[x for i,x in enumerate(distlst['Ann-ppt']) if neglst['Ann-ppt'][i] != 0])
a3.set_title('C',y=.9,loc='right',fontsize=20,fontweight='bold')
a3.spines['right'].set_visible(False)
a3.spines['top'].set_visible(False)
a3.yaxis.set_ticks_position('left')
a3.xaxis.set_ticks_position('bottom')

a4 = plt.subplot(334)
a4.scatter([x for x in neglst['CEC'] if x != 0],[x for i,x in enumerate(distlst['CEC']) if neglst['CEC'][i] != 0])
a4.set_title('D',y=.9,loc='right',fontsize=20,fontweight='bold')
a4.spines['right'].set_visible(False)
a4.spines['top'].set_visible(False)
a4.yaxis.set_ticks_position('left')
a4.xaxis.set_ticks_position('bottom')

a5 = plt.subplot(335)
a5.scatter([x for x in neglst['Clay'] if x != 0],[x for i,x in enumerate(distlst['Clay']) if neglst['Clay'][i] != 0])
a5.set_title('E',y=.9,loc='right',fontsize=20,fontweight='bold')
a5.spines['right'].set_visible(False)
a5.spines['top'].set_visible(False)
a5.yaxis.set_ticks_position('left')
a5.xaxis.set_ticks_position('bottom')

a6 = plt.subplot(336)
a6.scatter([x for x in neglst['Elev'] if x != 0],[x for i,x in enumerate(distlst['Elev']) if neglst['Elev'][i] != 0])
a6.set_title('F',y=.9,loc='right',fontsize=20,fontweight='bold')
a6.spines['right'].set_visible(False)
a6.spines['top'].set_visible(False)
a6.yaxis.set_ticks_position('left')
a6.xaxis.set_ticks_position('bottom')

a7 = plt.subplot(337)
a7.scatter([x for x in neglst['GDD-Aug'] if x != 0],[x for i,x in enumerate(distlst['GDD-Aug']) if neglst['GDD-Aug'][i] != 0])
a7.set_title('G',y=.9,loc='right',fontsize=20,fontweight='bold')
a7.spines['right'].set_visible(False)
a7.spines['top'].set_visible(False)
a7.yaxis.set_ticks_position('left')
a7.xaxis.set_ticks_position('bottom')

a8 = plt.subplot(338)
a8.scatter([x for x in neglst['GDD-May'] if x != 0],[x for i,x in enumerate(distlst['GDD-May']) if neglst['GDD-May'][i] != 0])
a8.set_title('H',y=.9,loc='right',fontsize=20,fontweight='bold')
a8.spines['right'].set_visible(False)
a8.spines['top'].set_visible(False)
a8.yaxis.set_ticks_position('left')
a8.xaxis.set_ticks_position('bottom')

a9 = plt.subplot(339)
a9.scatter([x for x in neglst['Lat'] if x != 0],[x for i,x in enumerate(distlst['Lat']) if neglst['Lat'][i] != 0])
a9.set_title('I',y=.9,loc='right',fontsize=20,fontweight='bold')
a9.spines['right'].set_visible(False)
a9.spines['top'].set_visible(False)
a9.yaxis.set_ticks_position('left')
a9.xaxis.set_ticks_position('bottom')

fig.set_size_inches(13,10)
#pdf.savefig(fig,bbox_inches='tight')

In [0]:
with PdfPages('/home/lindb/wbp/figures/log_neg_dij_v_envdist_sameenv_nozeros_AWSO-Lat.pdf') as pdf:
    plt.close('all')
    fig, ((a1,a2,a3),(a4,a5,a6),(a7,a8,a9)) = plt.subplots(3, 3, figsize=(5,5),dpi=400)

    a1 = plt.subplot(331)
    a1.scatter([math.log(-1*x,10) for x in neglst['AWS0-25'] if x != 0],[x for i,x in enumerate(distlst['AWS0-25']) if neglst['AWS0-25'][i] != 0])
    a1.set_title('A',y=.9,loc='right',fontsize=20,fontweight='bold')
    a1.spines['right'].set_visible(False)
    a1.spines['top'].set_visible(False)
    a1.yaxis.set_ticks_position('left')
    a1.xaxis.set_ticks_position('bottom')
    
    a2 = plt.subplot(332)
    a2.scatter([math.log(-1*x,10) for x in neglst['AWS0-50'] if x != 0],[x for i,x in enumerate(distlst['AWS0-50']) if neglst['AWS0-50'][i] != 0])
    a2.set_title('B',y=.9,loc='right',fontsize=20,fontweight='bold')
    a2.spines['right'].set_visible(False)
    a2.spines['top'].set_visible(False)
    a2.yaxis.set_ticks_position('left')
    a2.xaxis.set_ticks_position('bottom')
    
    a3 = plt.subplot(333)
    a3.scatter([math.log(-1*x,10) for x in neglst['Ann-ppt'] if x != 0],[x for i,x in enumerate(distlst['Ann-ppt']) if neglst['Ann-ppt'][i] != 0])
    a3.set_title('C',y=.9,loc='right',fontsize=20,fontweight='bold')
    a3.spines['right'].set_visible(False)
    a3.spines['top'].set_visible(False)
    a3.yaxis.set_ticks_position('left')
    a3.xaxis.set_ticks_position('bottom')
    
    a4 = plt.subplot(334)
    a4.scatter([math.log(-1*x,10) for x in neglst['CEC'] if x != 0],[x for i,x in enumerate(distlst['CEC']) if neglst['CEC'][i] != 0])
    a4.set_title('D*',y=.9,loc='right',fontsize=20,fontweight='bold')
    a4.spines['right'].set_visible(False)
    a4.spines['top'].set_visible(False)
    a4.yaxis.set_ticks_position('left')
    a4.xaxis.set_ticks_position('bottom')
    
    a5 = plt.subplot(335)
    a5.scatter([math.log(-1*x,10) for x in neglst['Clay'] if x != 0],[x for i,x in enumerate(distlst['Clay']) if neglst['Clay'][i] != 0])
    a5.set_title('E',y=.9,loc='right',fontsize=20,fontweight='bold')
    a5.spines['right'].set_visible(False)
    a5.spines['top'].set_visible(False)
    a5.yaxis.set_ticks_position('left')
    a5.xaxis.set_ticks_position('bottom')
    
    a6 = plt.subplot(336)
    a6.scatter([math.log(-1*x,10) for x in neglst['Elev'] if x != 0],[x for i,x in enumerate(distlst['Elev']) if neglst['Elev'][i] != 0])
    a6.set_title('F',y=.9,loc='right',fontsize=20,fontweight='bold')
    a6.spines['right'].set_visible(False)
    a6.spines['top'].set_visible(False)
    a6.yaxis.set_ticks_position('left')
    a6.xaxis.set_ticks_position('bottom')
    
    a7 = plt.subplot(337)
    a7.scatter([math.log(-1*x,10) for x in neglst['GDD-Aug'] if x != 0],[x for i,x in enumerate(distlst['GDD-Aug']) if neglst['GDD-Aug'][i] != 0])
    a7.set_title('G',y=.9,loc='right',fontsize=20,fontweight='bold')
    a7.spines['right'].set_visible(False)
    a7.spines['top'].set_visible(False)
    a7.yaxis.set_ticks_position('left')
    a7.xaxis.set_ticks_position('bottom')
    
    a8 = plt.subplot(338)
    a8.scatter([math.log(-1*x,10) for x in neglst['GDD-May'] if x != 0],[x for i,x in enumerate(distlst['GDD-May']) if neglst['GDD-May'][i] != 0])
    a8.set_title('H',y=.9,loc='right',fontsize=20,fontweight='bold')
    a8.spines['right'].set_visible(False)
    a8.spines['top'].set_visible(False)
    a8.yaxis.set_ticks_position('left')
    a8.xaxis.set_ticks_position('bottom')
    
    a9 = plt.subplot(339)
    a9.scatter([math.log(-1*x,10) for x in neglst['Lat'] if x != 0],[x for i,x in enumerate(distlst['Lat']) if neglst['Lat'][i] != 0])
    a9.set_title('I',y=.9,loc='right',fontsize=20,fontweight='bold')
    a9.spines['right'].set_visible(False)
    a9.spines['top'].set_visible(False)
    a9.yaxis.set_ticks_position('left')
    a9.xaxis.set_ticks_position('bottom')
    
    fig.set_size_inches(13,10)
    pdf.savefig(fig,bbox_inches='tight')

In [0]:
with PdfPages('/home/lindb/wbp/figures/log_pos_dij_v_envdist_sameenv_nozeros_AWSO-Lat.pdf') as pdf:
    plt.close('all')
    fig, ((a1,a2,a3),(a4,a5,a6),(a7,a8,a9)) = plt.subplots(3, 3, figsize=(5,5),dpi=400)

    a1 = plt.subplot(331)
    a1.scatter([math.log(x,10) for x in poslst['AWS0-25'] if x != 0],[x for i,x in enumerate(distlst['AWS0-25']) if poslst['AWS0-25'][i] != 0])
    a1.set_title('A',y=.9,loc='right',fontsize=20,fontweight='bold')
    a1.spines['right'].set_visible(False)
    a1.spines['top'].set_visible(False)
    a1.yaxis.set_ticks_position('left')
    a1.xaxis.set_ticks_position('bottom')
    
    a2 = plt.subplot(332)
    a2.scatter([math.log(x,10) for x in poslst['AWS0-50'] if x != 0],[x for i,x in enumerate(distlst['AWS0-50']) if poslst['AWS0-50'][i] != 0])
    a2.set_title('B',y=.9,loc='right',fontsize=20,fontweight='bold')
    a2.spines['right'].set_visible(False)
    a2.spines['top'].set_visible(False)
    a2.yaxis.set_ticks_position('left')
    a2.xaxis.set_ticks_position('bottom')
    
    a3 = plt.subplot(333)
    a3.scatter([math.log(x,10) for x in poslst['Ann-ppt'] if x != 0],[x for i,x in enumerate(distlst['Ann-ppt']) if poslst['Ann-ppt'][i] != 0])
    a3.set_title('C',y=.9,loc='right',fontsize=20,fontweight='bold')
    a3.spines['right'].set_visible(False)
    a3.spines['top'].set_visible(False)
    a3.yaxis.set_ticks_position('left')
    a3.xaxis.set_ticks_position('bottom')
    
    a4 = plt.subplot(334)
    a4.scatter([math.log(x,10) for x in poslst['CEC'] if x != 0],[x for i,x in enumerate(distlst['CEC']) if poslst['CEC'][i] != 0])
    a4.set_title('D*',y=.9,loc='right',fontsize=20,fontweight='bold')
    a4.spines['right'].set_visible(False)
    a4.spines['top'].set_visible(False)
    a4.yaxis.set_ticks_position('left')
    a4.xaxis.set_ticks_position('bottom')
    
    a5 = plt.subplot(335)
    a5.scatter([math.log(x,10) for x in poslst['Clay'] if x != 0],[x for i,x in enumerate(distlst['Clay']) if poslst['Clay'][i] != 0])
    a5.set_title('E',y=.9,loc='right',fontsize=20,fontweight='bold')
    a5.spines['right'].set_visible(False)
    a5.spines['top'].set_visible(False)
    a5.yaxis.set_ticks_position('left')
    a5.xaxis.set_ticks_position('bottom')
    
    a6 = plt.subplot(336)
    a6.scatter([math.log(x,10) for x in poslst['Elev'] if x != 0],[x for i,x in enumerate(distlst['Elev']) if poslst['Elev'][i] != 0])
    a6.set_title('F',y=.9,loc='right',fontsize=20,fontweight='bold')
    a6.spines['right'].set_visible(False)
    a6.spines['top'].set_visible(False)
    a6.yaxis.set_ticks_position('left')
    a6.xaxis.set_ticks_position('bottom')
    
    a7 = plt.subplot(337)
    a7.scatter([math.log(x,10) for x in poslst['GDD-Aug'] if x != 0],[x for i,x in enumerate(distlst['GDD-Aug']) if poslst['GDD-Aug'][i] != 0])
    a7.set_title('G',y=.9,loc='right',fontsize=20,fontweight='bold')
    a7.spines['right'].set_visible(False)
    a7.spines['top'].set_visible(False)
    a7.yaxis.set_ticks_position('left')
    a7.xaxis.set_ticks_position('bottom')
    
    a8 = plt.subplot(338)
    a8.scatter([math.log(x,10) for x in poslst['GDD-May'] if x != 0],[x for i,x in enumerate(distlst['GDD-May']) if poslst['GDD-May'][i] != 0])
    a8.set_title('H',y=.9,loc='right',fontsize=20,fontweight='bold')
    a8.spines['right'].set_visible(False)
    a8.spines['top'].set_visible(False)
    a8.yaxis.set_ticks_position('left')
    a8.xaxis.set_ticks_position('bottom')
    
    a9 = plt.subplot(339)
    a9.scatter([math.log(x,10) for x in poslst['Lat'] if x != 0],[x for i,x in enumerate(distlst['Lat']) if poslst['Lat'][i] != 0])
    a9.set_title('I',y=.9,loc='right',fontsize=20,fontweight='bold')
    a9.spines['right'].set_visible(False)
    a9.spines['top'].set_visible(False)
    a9.yaxis.set_ticks_position('left')
    a9.xaxis.set_ticks_position('bottom')
    
    fig.set_size_inches(13,10)
    pdf.savefig(fig,bbox_inches='tight')

In [0]:
for env in poslst.keys():
    spear = spearmanr([math.log(x,10) for x in poslst[env] if x != 0],[x for i,x in enumerate(distlst[env]) if poslst[env][i] != 0])
    if spear[1] <= 0.05:
        print env,spear

# effect size distributions

In [0]:
#file made in 07_hierfstat.ipynb
filE = '/home/lindb/wbp/hierfstat/imputed/imputed_hierarchical_Fstats.txt'
df = pd.read_csv(filE,header=0,index_col=0,sep='\t')
df.head()

In [0]:
#get bayenv2 snps
DIR = '/home/lindb/wbp/bayenv2/results/sigsnps'
bayfs = [op.join(DIR,f) for f in ls(DIR)]
baydict = {}
for f in bayfs:
    env = op.basename(f).split("_")[0]
    df = pd.read_csv(f,header=0,sep='\t')
    baydict[env] = df[env].tolist()
    print env,len(baydict[env])

In [0]:
for env in baydict:
    with PdfPages('/home/lindb/wbp/figures/multilocus_fst_distribution_bayenv_%s_overlay.pdf' % env) as pdf:
        plt.close('all')
        fig  = plt.figure(figsize=(5,5),dpi=400)
        a1 = plt.subplot(111)
        a1.hist(df['plot_total'],bins = [x for x in np.linspace(-0.1,0.2,30)])
        a1.spines['right'].set_visible(False)
        a1.spines['top'].set_visible(False)
        a1.yaxis.set_ticks_position('left')
        a1.xaxis.set_ticks_position('bottom')
        a1.set_xlabel(r'multilocus $F_{ST}$',fontsize=14)
        a1.set_ylabel('Count',fontsize=14)

        #[a1.axvline(x=dd[snp],c='red',linewidth=0.25,zorder=1) for snp in dd.keys()]
        #a1.axvline(x=dd[dd.keys()[0]],c='red',linewidth=0.25,zorder=1)

        fig.set_size_inches(4,4)
        pdf.savefig(fig,bbox_inches='tight')

In [0]:
envdf

In [0]:
from scipy.stats import spearmanr

In [0]:
from scipy.stats import pearsonr

In [0]:
for i,envi in enumerate(envdf.index):
    for j,envj in enumerate(envdf.index):
        if i > j:
            corr,p = spearmanr(envdf.loc[envi,:],envdf.loc[envj,:])
            if p <= 0.05:
                print envi,envj,corr,p

In [0]:
for i,envi in enumerate(envdf.index):
    for j,envj in enumerate(envdf.index):
        if i > j:
            corr,p = pearsonr(envdf.loc[envi,:],envdf.loc[envj,:])
            if p <= 0.05:
                print envi,envj,corr,p

In [0]:
spearmanr(envdf.loc['Ann-ppt',:],envdf.loc['Elev',:])

In [0]:
p

In [0]:
corr

# see if correlations are driven by larger exp het

In [0]:
#file made in 09_OutFLANK.ipynb
globmafs = pd.read_csv('/home/lindb/wbp/OutFLANK/global_mafs.txt',header=0,sep='\t')
globmafs.head()

In [0]:
popmafs = pd.read_csv('/home/lindb/wbp/OutFLANK/imputed_MAF.txt',header=0,index_col=0,sep='\t')
popmafs.head()

In [0]:
popmafs.loc['NODE_1000013_length_91_cov_1.802198_37',:].tolist()

In [0]:
envdf = pd.read_csv('/home/lindb/wbp/bayenv2/ENVIRONFILE_headerIDX.txt',header=0,index_col=0,sep='\t')
envdf.head()

In [0]:
from scipy.stats import spearmanr

In [0]:
#get env correlations for each snp
#rows = snps, cols = envs
snpDict = OrderedDict()
snpcount = 0
for snp in popmafs.index:
    snpDict[snp] = OrderedDict()
    freqs = popmafs.loc[snp,:].tolist()
    for env in envdf.index:
        envval = envdf.loc[env,:].tolist()
        s = spearmanr(freqs,envval)
        snpDict[snp][env] = s[0]
    snpcount += 1
    if snpcount % 1000 == 0:
        print snpcount

In [0]:
#write out the file
key0 = snpDict.keys()[0]
filE = '/home/lindb/wbp/bayenv2/uncorrected_correlations.txt'
with open(filE,'w') as o:
    text = '\t'.join(snpDict[key0].keys())+'\n'
    o.write("%s" % text)
    for snp in snpDict:
        text = snp+'\t'+'\t'.join([str(x) for x in snpDict[snp].values()])+'\n'
        o.write("%s" % text)

In [0]:
corr = pd.read_csv(filE,header=0,index_col=0,sep='\t')
corr.head()

In [0]:
#get number of bayenv sig SNPs
DIR = '/home/lindb/wbp/bayenv2/results/sigsnps'
bayDict = {}
for f in [op.join(DIR,f) for f in ls(DIR)]:
    env = op.basename(f).split("_")[0]
    df = pd.read_csv(f,header=0,sep='\t')
    bayDict[env] = len(df.index)
    print env,len(df.index)

In [0]:
#get absolute rhos
abscorr = pd.DataFrame(abs(corr))
abscorr.head()

In [0]:
abscorr.sort_values(by='AWS0-25',inplace=True,ascending=0)
abscorr.head()

In [0]:
abscorr.loc[abscorr.index[0:100],:]

In [0]:
freqs = popmafs.loc['NODE_768610_length_47_cov_4.042553_36',:].tolist()
envval = envdf.loc['AWS0-25',:].tolist()
plt.scatter(freqs,envval)

In [0]:
corrdict = {}
for env in abscorr:
    abscorr.sort_values(by=env,inplace=True,ascending=0)
    corrdict[env] = list(abscorr.index[0:bayDict[env]])
    print len(corrdict[env]),bayDict[env]

In [0]:
#get expected heterozygosities
filE = '/home/lindb/wbp/OutFLANK/Hexp_by_snp_withbins.txt'
H = pd.read_csv(filE,header=0,index_col=0,sep='\t')
H.head()

In [0]:
np.unique(H['bin'])

In [0]:
Habscorr = pd.merge(pd.DataFrame(H['bin']),abscorr,left_index=True,right_index=True)
Habscorr.head()

In [0]:
abscorr.sort_values(by='AWS0-25',ascending=1,inplace=True)
abscorr.head()

In [0]:
#make scatters 
for env in abscorr:
    print env
    abscorr.sort_values(by=env,ascending=0,inplace=True)
    f00 = pd.DataFrame(abscorr[env][abscorr.index[0:500]])
    H00 = pd.DataFrame(Habscorr['bin'][H.index.isin(f00.index)])
    plt.scatter(f00[env].tolist(),H00['bin'].tolist())
    plt.show()

In [0]:
#make histograms of Hexp for SNPs correlated to env without correction for structure
for env in corrdict:
    df = pd.DataFrame(H[H.index.isin(corrdict[env])])
    print env
    plt.hist(df['bin'],bins=np.arange(0,51,1))
    plt.show()

In [0]:
#make box plots
ulims = [10,20,30,40,50]
llims = [0,10,20,30,40]
for env in corrdict:
    df = pd.DataFrame(H[H.index.isin(corrdict[env])]) #get bins
    df1 = pd.DataFrame(abscorr[env]) #get rhos
    df = pd.merge(df1,df,left_index=True,right_index=True)
    boxes=[]
    count = 0
    for lim in ulims:
        df2 = pd.DataFrame(df[df['bin']<ulims[count]])
        df3 = pd.DataFrame(df2[df2['bin']>=llims[count]])
        boxes.append(df3[env].tolist())
        count += 1
    print env,len(corrdict[env])
    plt.boxplot(boxes)
    plt.show()

In [0]:
df.head()

In [0]:
boxes[0].head()

# bf.environfile - get the sig SNPs - this was done with the first chain only (ignore this section)

In [0]:
#get a DF that contains the names for environmental variables
envdf = pd.read_csv('/home/lindb/wbp/bayenv2/ENVIRONFILE_headerIDX.txt',header=0,index_col=0,sep='\t')
envdf.head()

In [0]:
len(envdf.index)

In [0]:
#assign column value of the bf.environfile.ENVIRONFILE for each environmental variable
envDict = OrderedDict()
for i, env in enumerate(envdf.index):
    #print i,env
    envDict[env] = [(i*3),(i*3)+1,(i*3)+2]
envDict

In [0]:
#get a dictionary to name columns in bf_environfile.ENVIRONFILE
orderDict = {}
orderDict[0] = 'BF'
orderDict[1] = 'rho'
orderDict[2] = 'pearson'
colDict = OrderedDict()
for env,cols in envDict.items():
    #print env,cols
    colCount = 0
    for col in cols:
        sub = '_'.join([str(env), orderDict[colCount]]) 
        colDict[col] = sub
        colCount += 1

In [0]:
#transform the bf_environfile to have indices and column names
df = pd.read_csv('/home/lindb/wbp/bayenv2/bf_environfile.ENVIRONFILE.txt.bf',header=None,sep='\t')s
loci = []
for row in df.index:
    locus = df.loc[row,0].split("/")[-1]
    loci.append(locus)
bayenv2 = df.loc[:,[x for x in df.columns[1:-1]]] #get rid of locus column and the 'blank' column at the end
bayenv2.columns = [col for col in range(len(bayenv2.columns))] #rename columns from 0 to len(columns)
bayenv2.columns = [colDict[col] for col in bayenv2.columns] #convert to BF, rho, pearson
bayenv2.index = [locus for locus in loci]
bayenv2.head()

In [0]:
pd.DataFrame(bayenv2.loc['NODE_335159_length_90_cov_1.000000_19',:]).T

In [0]:
len(np.unique(bayenv2.index).tolist())

In [0]:
#get a rho df
rhos = [col for col in bayenv2.columns if 'rho' in col]
rhodf = bayenv2.loc[:,[col for col in rhos]]
filE = '/home/lindb/wbp/bayenv2/results/rho_all.txt'
rhodf.to_csv(filE,header=True,index=True,sep='\t')
rhodf.head()

In [0]:
#get absolute values of rhos to rank them
max(abs(rhodf['AWS0-25_rho']))

In [0]:
len(rhodf.index)

In [0]:
#get a dictionary with the 0.01% (1162) largest rhos for each env
rho2 = pd.DataFrame(abs(rhodf))
rhoDict = OrderedDict() #k = env, val = list(top_loci)
for env in rho2.columns:
    sorts = pd.DataFrame(rho2.sort_values(by=env,ascending=0))
    rhoDict[env] = sorts.index[:1162] #top loci

In [0]:
#get a BF df
bfs = [col for col in bayenv2.columns if '_BF' in col]
bfdf = bayenv2.loc[:,[col for col in bfs]]
filE = '/home/lindb/wbp/bayenv2/results/BF_all.txt'
bfdf.to_csv(filE,header=True,index=True,sep='\t')
bfdf.head()

In [0]:
bfdf = pd.read_csv('/home/lindb/wbp/bayenv2/results/BF_all.txt',header=0,index_col=0,sep='\t')
bfdf.head()

In [0]:
#get a dictionary with the 0.01% (1162) largest BFs for each env
bf2 = pd.DataFrame(abs(bfdf))
bfDict = OrderedDict()
for env in bf2.columns:
    sorts = pd.DataFrame(bf2.sort_values(by=env,ascending=0))
    bfDict[env] = sorts.index[:1162]

In [0]:
#get the intersection of loci between top rhos and top BFs
intDict = OrderedDict()
for env in envDict.keys():
    bfkey = env+'_BF'
    rhokey = env+'_rho'
    intDict[env] = list(set(bfDict[bfkey]).intersection(set(rhoDict[rhokey])))

In [0]:
#see how many 'significant' SNPs for each env variable
for env in intDict.keys():
    print env,len(intDict[env])

In [0]:
min([len(intDict[env]) for env in intDict.keys()]),max([len(intDict[env]) for env in intDict.keys()])

In [0]:
#write out files
for env in intDict.keys():
    filE = '/home/lindb/wbp/bayenv2/results/rhoBF_intersection_%s.txt' % env
    df = pd.DataFrame(intDict[env])
    df.columns = [env]
    df.to_csv(filE,header=True,index=False)
df.head()

In [0]:
intDict.keys()

# Get BFs >= 3

In [0]:
bfdf.head()

In [0]:
#get the loci to run bayenv2 null runs
nullloci = [] #keep track of the loci
best = []
worst = []
for envbf in bfdf.columns:
    #get all BFs greater than or equal to 3
    env = envbf.split("_")[0]
    data = pd.DataFrame(bfdf[envbf])
    new = pd.DataFrame(data[data[envbf] >= 3] )
    [best.append(locus) for locus in new.index if locus not in best]
    
    #get 10 of the worst BFs
    bad    = pd.DataFrame(data[data[envbf]<=3])
    badder = pd.DataFrame(bad.sort_values(by=envbf,ascending=1))
    badder = badder[badder.index.isin(badder.index[:10])]
    [worst.append(locus) for locus in badder.index if locus not in worst]
    
    #concat the two dataframes
    catd = pd.concat([badder,new])
    catd.sort_values(by=envbf,ascending=1,inplace=True)
    
    #add loci to nullloci list
    [nullloci.append(locus) for locus in catd.index if locus not in nullloci]
    
    filE = '/home/lindb/wbp/bayenv2/results/%s_bfgreaterthan3and10worst.txt' % env
    catd.to_csv(filE,header=True,index=True,sep='\t')
    
    
    print env,len(catd.index),len(new.index),len(badder.index),len(bad.index)

In [0]:
new.sort_values(by='WC3rdbar_BF',ascending=1).head()

In [0]:
catd.head()

In [0]:
len(new.index)

In [0]:
len(best),len(worst),len(nullloci)

In [0]:
nulls = pd.DataFrame(nullloci)
nulls.columns = ['topandlowestBFs']
filE = '/home/lindb/wbp/bayenv2/results/uniq_loci_bfgreaterthan3and10worst.txt'
nulls.to_csv(filE,header=True,index=False,sep='\t')
nulls.head()

In [0]:
len(nulls)

In [0]:
best = pd.DataFrame(best)
best.columns = ['BF>=3']
filE = '/home/lindb/wbp/bayenv2/results/best_loci_bfgreaterthan3without10worst.txt'
best.to_csv(filE,header=True,index=False,sep='\t')
best.head()

In [0]:
worst = pd.DataFrame(worst)
worst.columns = ['worstBF']
filE = '/home/lindb/wbp/bayenv2/results/worst_loci_bfgreaterthan3without10worst.txt'
worst.to_csv(filE,header=True,index=False,sep='\t')
worst.head()